## 1) Setup (imports + paths)

Why this step is necessary:
- We need standard tooling to load CSVs, profile columns, and export processed datasets.
- We use `pathlib.Path` to keep file handling robust and portable.

No data is changed here—this only prepares the environment.

In [ ]:
import re

from pathlib import Path



import numpy as np

import pandas as pd



RANDOM_STATE = 42

np.random.seed(RANDOM_STATE)



pd.set_option('display.max_columns', 250)

pd.set_option('display.width', 160)

pd.set_option('display.max_rows', 200)

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')



# Project folders

project_root = Path('.').resolve()

data_dir = project_root  # kept for backwards-compatibility in older cells



raw_dir = project_root / 'data' / 'raw'

intermediate_dir = project_root / 'data' / 'intermediate'

processed_dir = project_root / 'data' / 'processed'



splits_dir = project_root / 'data' / 'outputs' / 'splits'

grouped_dir = project_root / 'data' / 'outputs' / 'grouped'

grouped_imputed_dir = project_root / 'data' / 'outputs' / 'grouped_imputed'



for d in [raw_dir, intermediate_dir, processed_dir, splits_dir, grouped_dir, grouped_imputed_dir]:

    d.mkdir(parents=True, exist_ok=True)



# Accept either the organized location (data/raw) or old root location.

pred_candidates = [

    raw_dir / 'Copy of 2024-2025-Ensia Performance prediction.csv',

    raw_dir / 'Copy of 2024-2025-Ensia Performance prediction - Copy of 2024-2025-Ensia Performance prediction.csv',

    raw_dir / 'Copy of 2024-2025-Ensia Performance prediction.xlsx',

    data_dir / 'Copy of 2024-2025-Ensia Performance prediction.csv',

    data_dir / 'Copy of 2024-2025-Ensia Performance prediction - Copy of 2024-2025-Ensia Performance prediction.csv',

    data_dir / 'Copy of 2024-2025-Ensia Performance prediction.xlsx',

]

scores_candidates = [

    raw_dir / 'Final_students_data_scores.csv',

    raw_dir / 'Copy of Final_students_data_scores.xlsx - Final_students_data_scores.csv',

    raw_dir / 'Copy of Final_students_data_scores.xlsx',

    data_dir / 'Final_students_data_scores.csv',

    data_dir / 'Copy of Final_students_data_scores.xlsx - Final_students_data_scores.csv',

    data_dir / 'Copy of Final_students_data_scores.xlsx',

]



def first_existing(paths: list[Path]) -> Path:

    for p in paths:

        if p.exists():

            return p

    raise FileNotFoundError('None of the expected input files exist: ' + ', '.join(str(p) for p in paths))



path_pred = first_existing(pred_candidates)

path_scores = first_existing(scores_candidates)



print('Project root:', project_root)

print('Raw dir:', raw_dir)

print('Processed dir:', processed_dir)

print('Outputs dir:', (project_root / 'data' / 'outputs'))

print('Using dataset 1:', path_pred)

print('Using dataset 2:', path_scores)


Using dataset 1: Copy of 2024-2025-Ensia Performance prediction - Copy of 2024-2025-Ensia Performance prediction.csv
Using dataset 2: Copy of Final_students_data_scores.xlsx - Final_students_data_scores.csv


## 2) Load both CSV datasets (read-only)

Why this step is necessary:
- ENSIA project data is split across multiple sources (survey + scores).
- We load them **as-is** first so we can diagnose issues before doing any cleaning.

Notes:
- These CSVs contain long header strings and sometimes newlines inside quoted headers; we use `engine='python'` to be more tolerant.
- We keep the raw DataFrames separate (`df_pred_raw`, `df_scores_raw`) for transparent comparison.

In [255]:
def read_csv_robust(path: Path) -> pd.DataFrame:
    # engine='python' tends to be more tolerant for complex quoting/newlines in headers
    return pd.read_csv(path, engine='python')

df_pred_raw = read_csv_robust(path_pred)
df_scores_raw = read_csv_robust(path_scores)

print('df_pred_raw shape:', df_pred_raw.shape)
print('df_scores_raw shape:', df_scores_raw.shape)

# -------------------------
# Validation checks (catch issues early)
# -------------------------
if df_pred_raw.shape[0] == 0 or df_scores_raw.shape[0] == 0:
    raise ValueError('One of the input datasets is empty. Check the CSV exports and paths.')

if df_pred_raw.shape[1] == 0 or df_scores_raw.shape[1] == 0:
    raise ValueError('One of the input datasets has 0 columns. Check CSV formatting / delimiter / quoting.')

# pandas can hold duplicate column names; this breaks many operations later
if not df_pred_raw.columns.is_unique:
    print('WARNING: df_pred_raw has duplicate column names (will be collapsed/standardized in the next step).')
if not df_scores_raw.columns.is_unique:
    print('WARNING: df_scores_raw has duplicate column names (will be collapsed/standardized in the next step).')

display(df_pred_raw.head())
display(df_scores_raw.head())

df_pred_raw shape: (200, 115)
df_scores_raw shape: (275, 61)


,Timestamp,What is your gender :,In which year of your studies are you currently enrolled:,What was your BAC specialty :,How many hours per week do you spend studying (personnal efforts):,Where do you prefer to study:,You prefer to study:,What learning methods do help you understand better:,What types of resources do you use most:,Do you use a planer or a schedule in your revisions :,For the next 8 questions you will have a list of problems (obstacles) classify them according to how much they influence your studies:\n1. First :,2. Second :,3. Third :,4. Fourth :,5. Fifth :,6. Sixth :,7. Seventh :,8. Eighth :,Other problems that you face:,Rate your satisfaction with the school's program:,Do you engage in any external activities (not related to the study field):,Your level of motivation for joining ENSIA:,How much do you like the field of Artificial Intelligence :,How do other's feedback influence you:,How many hours do you sleep :,How well do you handle stress and pressure :,In a scale of 1 to 10 how do you rate your level in the following fields:\n1. English:,2. Programming:,3. Computer science:,4. Mathematics:,1Y_Average S1,1Y_Average S2,1Y_Average,2Y_Average S1,2Y_Average S2,2Y_Average,3Y_Average S1,3Y_Average S2,3Y_Average,1Y_Foundational Mathematics,1Y_DSA1,1Y_Digital Systems,1Y_Information Technology Essentials,1Y_English 1,1Y_CPSCT,1Y_OOP,1Y_Introduction to Linux,1Y_Linear Algebra,1Y_Mathematical Analysis 1,1Y_Introduction to Statistics,1Y_English 2,2Y_DSA 2,2Y_Logic,2Y_Mathematical Analysis 2,2Y_Databases,2Y_Probability,2Y_Web Development,2Y_Introduction to Business,2Y_Theory of Computing,2Y_Operating Systems,2Y_Computer Architecture,2Y_Statistical Inference,2Y_Introduction to AI,2Y_Mathematical Analysis 3,2Y_Electronic Circuits Labs,3Y_Data Mining,3Y_Operations Research,3Y_Stochastic,3Y_Software Engineering,3Y_Networks and Protocols,3Y_Mobile Development,3Y_Entrepreneurship and Innovation,3Y_Machine Learning,3Y_Numerical Methods and Optimisation,3Y_Time Series,3Y_Advanced Databases,3Y_Computer and network Security,1Y_Foundational Mathematics.1,1Y_DSA1.1,1Y_Digital Systems.1,1Y_Information Technology Essentials.1,1Y_English 1.1,1Y_CPSCT.1,1Y_OOP.1,1Y_Introduction to Linux.1,1Y_Linear Algebra.1,1Y_Mathematical Analysis 1.1,1Y_Introduction to Statistics.1,1Y_English 2.1,2Y_DSA 2.1,2Y_Logic.1,2Y_Mathematical Analysis 2.1,2Y_Databases.1,2Y_Probability.1,2Y_Web Development.1,2Y_Introduction to Business.1,2Y_Theory of Computing.1,2Y_Operating Systems.1,2Y_Computer Architecture.1,2Y_Statistical Inference.1,2Y_Introduction to AI.1,2Y_Mathematical Analysis 3.1,2Y_Electronic Circuits Labs.1,1Y_Foundational Mathematics.2,1Y_DSA1.2,1Y_Digital Systems.2,1Y_Information Technology Essentials.2,1Y_English 1.2,1Y_CPSCT.2,1Y_OOP.2,1Y_Introduction to Linux.2,1Y_Linear Algebra.2,1Y_Mathematical Analysis 1.2,1Y_Introduction to Statistics.2,1Y_English 2.2
0,2025/01/08 6:00:12 PM GMT+1,Female,First year,Science,18-25,In your room (dorms),Alone,Learning by practicing,Online tutorials,Yes,Pressure,Bad quality of internet,Lack of time,Lack of previous knowledge or experience in th...,Teachers' teaching method,Not having a good (powerful) computer,Teachers' teaching method,Private life problems,NaN,3,Yes,4,5,Positively,5-7 hours,3,9,8,8,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025/01/08 6:00:16 PM GMT+1,Male,First year,Science,10-17,In your room (dorms),Alone,Lectures;Learning by practicing;Mind mapping a...,Online documents;YouTube videos,No,Teachers' teaching method,Lack of previous knowledge or experience in th...,Lack of time,Bad quality of internet,Teaching language,Pressure,Not having a good (powerful) computer,Private life problems,NaN,3,No,

,What is your gender :,In which year of your studies are you currently enrolled:,What was your BAC specialty :,How many hours per week do you spend studying (personnal efforts):,Where do you prefer to study:,You prefer to study:,What learning methods do help you understand better:,What types of resources do you use most:,Do you use a planer or a schedule in your revisions :,For the next 8 questions you will have a list of problems (obstacles) classify them according to how much they influence your studies:\r\n1. First :,2. Second:,3. Third:,4. Forth:,5. Fifth:,6. sixth:,7. seventh:,8. eighth,Rate your satisfaction with the school's program:,Do you engage in any external activities (not related to the study field):,Your level of motivation for joining ENSIA,How much do you like the field of Artificial Intelligence :,How do others' feedbacks influence you:,How many hours do you sleep,How well do you handle stress and pressure :,In a scale of 1 to 10 how do you rate your level in the following fields:\r\n1. English:,2. Programming:,3. Computer science:,4. Mathematics,1Y_Intensive english,1Y_Seminar,1Y_CPSCT,1Y_Analysis I,1Y_Algebra,1Y_A. Com. I,1Y_Average S1,1Y_Intro. Prog. C++,1Y_Analysis II,1Y_A. Com. II,1Y_OOP,1Y_Linear Algebra,1Y_Proba Stat,1Y_Average S2,1Y_Average,2Y_Analysis III,2Y_DSA,2Y_Archi 1,2Y_Logic,2Y_Complexity,2Y_Proba Stat II,2Y_Lin. Alg II,2Y_DataBases,2Y_Average S1,2Y_Intro AI,2Y_Oper. res,2Y_Archi II,2Y_Oper. Syst,2Y_Soft. Eng.,2Y_Web Prog.,2Y_Intro. Busin.,2Y_Average S2,2Y_Average
0,Male,First year,Mathematics,18-25,In your room (dorms),Alone,"Lectures, Learning by practicing, Project-base...","Online tutorials, Online documents, YouTube vi...",No,Pressure,Private life problems,Lack of time,Teaching language,Not having a good (powerful) computer,Bad quality of internet,Lack of previous knowledge or experience in th...,Teachers' teaching method,3,Yes,4,5,Negatively,5-7 hours,2,7,7,9,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,Third year,Mathematics,10-17,In your room (dorms),In peer,"Lectures, Learning by practicing, Project-base...","Online tutorials, Online documents",No,Teachers' teaching method,Lack of time,Lack of previous knowledge or experience in th...,Pressure,Bad quality of internet,Private life problems,Not having a good (powerful) computer,Teaching language,2,Yes,3,5,You do not care about,5-7 hours,3,8,8,8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Very High,High,Moderate,Moderate,High,High,High,High,High,Moderate,Moderate,Moderate,High,High,High,High,12.0000,13.0000
2,Male,First year,Science,18-25,In your room (dorms),Alone,Lectures,Online documents,Yes,Private life problems,Lack of time,Bad quality of internet,Pressure,Lack of previous knowledge or experience in th...,Teaching language,Teachers' teaching method,Not having a good (powerful) computer,3,Yes,3,4,You do not care about,5-7 hours,5,8,5,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Male,Third year,Mathematics,18-25,In your room (dorms),Alone,"Lectures, Learning by practicing, Project-base...","Online tutorials, YouTube videos",No,Bad quality of internet,Lack of previous knowledge or experience in th...,Teachers' teaching method,Lack of time,Not having a good (powerful) computer,Pressure,Teaching language,Private life problems,3,No,5,5,Positively,5-7 hours,4,7,8,8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Very High,High,Moderate,High,High,High,High,High,High,Moderate,High,High,High,High,High,High,12.0000,13.0000
4,Female,Third year,Mathematics,18-25,In your room (dorms),Alone,"Lectures, Learning by practicing, Mind mapping...","Books, Online documents, YouTube videos",No,Teachers' teaching method,Lack of previous knowledge or experience in th...,Pressure,Bad quality of internet,Private life problems,Lack of time,T

## 3) Fix duplicated columns + standardize column names (alignment only; no cleaning of values yet)

Why this step is necessary:
- Dataset 1 contains **repeated module columns** caused by the export (e.g., the same module appears multiple times).
  - When pandas reads this CSV, duplicates are shown as `Module`, `Module.1`, `Module.2`.
- Before we can merge and analyze, we must collapse those duplicates into **one column per module**.
- Then we standardize headers so the *same fields* align across the two datasets.

Step A — collapse duplicate module columns (Dataset 1 main issue):
- For each repeated group like `X`, `X.1`, `X.2`, we keep **one column named `X`**.
- Row-wise fill rule (your requirement):
  - If only one of the three columns is filled, keep that value.
  - If more than one is filled, we keep the first non-null and **report conflicts** for review.

Step B — standardize headers for alignment (safe operations on header strings only):
- Replace tabs/newlines with spaces; collapse repeated spaces
- Normalize to lowercase for matching
- Normalize punctuation consistently
- Make duplicates unique *and report collisions* so we don’t silently mix two different columns

Important:
- Step A changes the *schema* (removes duplicated columns) but does not modify the actual values.
- Step B does **not** change any data values—only the column labels used for alignment.

In [256]:
import unicodedata

def collapse_pandas_duplicate_columns(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Collapse duplicated columns like X, X.1, X.2 into one X by row-wise first-non-null.

    Returns: (df_collapsed, report_df)
    report_df includes conflict counts where >1 column was non-null and values disagreed.
    """
    cols = list(df.columns)

    def base_name(col: str) -> str:
        m = re.match(r'^(.*)\.(\d+)$', str(col))
        return m.group(1) if m else str(col)

    bases = [base_name(c) for c in cols]
    groups: dict[str, list[str]] = {}
    for c, b in zip(cols, bases):
        groups.setdefault(b, []).append(c)

    df_out = df.copy()
    report_rows = []

    # Process only bases that truly repeat
    repeated = {b: cs for b, cs in groups.items() if len(cs) > 1}
    for b, cs in repeated.items():
        # Preserve original left-to-right order as it appears in the CSV
        cs_in_order = [c for c in df_out.columns if c in cs]
        if len(cs_in_order) <= 1:
            continue

        pos = df_out.columns.get_loc(cs_in_order[0])
        block = df_out[cs_in_order]
        combined = block.bfill(axis=1).iloc[:, 0]
        nonnull_counts = block.notna().sum(axis=1)

        # Conflict = more than one non-null AND not all non-nulls equal to chosen combined value
        same_as_combined_or_na = block.eq(combined, axis=0) | block.isna()
        conflict_mask = (nonnull_counts > 1) & (~same_as_combined_or_na.all(axis=1))
        conflict_rows = int(conflict_mask.sum())
        any_value_rows = int((nonnull_counts > 0).sum())

        report_rows.append({
            'base_column': b,
            'n_columns_collapsed': len(cs_in_order),
            'rows_with_any_value': any_value_rows,
            'rows_with_conflict': conflict_rows,
            'columns': cs_in_order,
        })

        # Drop the whole group and re-insert single base column at the first position
        df_out = df_out.drop(columns=cs_in_order)
        df_out.insert(pos, b, combined)

    report_df = (
        pd.DataFrame(report_rows).sort_values(['rows_with_conflict', 'n_columns_collapsed'], ascending=False)
        if report_rows
        else pd.DataFrame(columns=['base_column','n_columns_collapsed','rows_with_any_value','rows_with_conflict','columns'])
    )
    return df_out, report_df

def canonicalize_column_name(name: str) -> str:
    """Create a canonical header used for matching across datasets."""
    s = str(name)
    s = unicodedata.normalize('NFKC', s)
    s = s.replace(chr(9), ' ').replace(chr(10), ' ')
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def make_unique(names: list[str]) -> list[str]:
    seen: dict[str, int] = {}
    out: list[str] = []
    for n in names:
        if n not in seen:
            seen[n] = 0
            out.append(n)
        else:
            seen[n] += 1
            out.append(f"{n}__dup{seen[n]}")
    return out

def standardize_headers_pair(df_a: pd.DataFrame, df_b: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Standardize headers for BOTH dataframes consistently + report collisions."""
    a_orig = list(df_a.columns)
    b_orig = list(df_b.columns)

    a_can = [canonicalize_column_name(c) for c in a_orig]
    b_can = [canonicalize_column_name(c) for c in b_orig]

    def collisions(orig: list[str], can: list[str], tag: str) -> list[dict]:
        tmp = pd.DataFrame({'dataset': tag, 'original': orig, 'canonical': can})
        g = tmp.groupby('canonical', dropna=False)['original'].apply(list)
        out = []
        for canon, originals in g.items():
            if len(originals) > 1:
                out.append({'dataset': tag, 'canonical': canon, 'n_originals': len(originals), 'originals': originals[:8]})
        return out

    coll = collisions(a_orig, a_can, 'dataset1') + collisions(b_orig, b_can, 'dataset2')
    collisions_df = (
        pd.DataFrame(coll).sort_values(['n_originals', 'canonical'], ascending=[False, True])
        if coll
        else pd.DataFrame(columns=['dataset', 'canonical', 'n_originals', 'originals'])
    )

    a_new = make_unique(a_can)
    b_new = make_unique(b_can)

    df_a_std = df_a.copy()
    df_b_std = df_b.copy()
    df_a_std.columns = a_new
    df_b_std.columns = b_new

    return df_a_std, df_b_std, collisions_df

# Step A: collapse duplicated columns caused by CSV export (Dataset 1 main issue)
df_pred_raw_collapsed, dup_report_pred = collapse_pandas_duplicate_columns(df_pred_raw)
df_scores_raw_collapsed, dup_report_scores = collapse_pandas_duplicate_columns(df_scores_raw)

print('Duplicate-column collapse report (dataset 1) — top 15:')
display(dup_report_pred.head(15))
print('Duplicate-column collapse report (dataset 2) — top 15:')
display(dup_report_scores.head(15))

print('Shapes before/after collapse:')
print('df_pred_raw:', df_pred_raw.shape, '->', df_pred_raw_collapsed.shape)
print('df_scores_raw:', df_scores_raw.shape, '->', df_scores_raw_collapsed.shape)

# Step B: standardize BOTH datasets consistently (header alignment)
df_pred, df_scores, collisions_df = standardize_headers_pair(df_pred_raw_collapsed, df_scores_raw_collapsed)

print('Standardized columns:')
print('df_pred cols:', len(df_pred.columns))
print('df_scores cols:', len(df_scores.columns))

pred_only = sorted(set(df_pred.columns) - set(df_scores.columns))
scores_only = sorted(set(df_scores.columns) - set(df_pred.columns))
common = sorted(set(df_pred.columns) & set(df_scores.columns))

print('Common columns (after standardization):', len(common))
print('Columns only in df_pred:', len(pred_only))
print('Columns only in df_scores:', len(scores_only))

if not collisions_df.empty:
    print('WARNING: header collisions detected (multiple original columns mapped to same canonical name).')
    display(collisions_df.head(20))
else:
    print('No header-collisions detected.')

orig_pred = set(map(str, df_pred_raw.columns))
orig_scores = set(map(str, df_scores_raw.columns))
raw_common = sorted(orig_pred & orig_scores)
print('Common columns BEFORE standardization (raw):', len(raw_common))

display(pd.DataFrame({'common_after_standardization_sample': common[:30]}))
display(pd.DataFrame({'df_pred_only_sample': pred_only[:30]}))
display(pd.DataFrame({'df_scores_only_sample': scores_only[:30]}))

Duplicate-column collapse report (dataset 1) — top 15:


,base_column,n_columns_collapsed,rows_with_any_value,rows_with_conflict,columns
1,1Y_DSA1,3,104,1,"[1Y_DSA1, 1Y_DSA1.1, 1Y_DSA1.2]"
5,1Y_CPSCT,3,100,1,"[1Y_CPSCT, 1Y_CPSCT.1, 1Y_CPSCT.2]"
0,1Y_Foundational Mathematics,3,103,0,"[1Y_Foundational Mathematics, 1Y_Foundational ..."
2,1Y_Digital Systems,3,87,0,"[1Y_Digital Systems, 1Y_Digital Systems.1, 1Y_..."
3,1Y_Information Technology Essentials,3,83,0,"[1Y_Information Technology Essentials, 1Y_Info..."
4,1Y_English 1,3,101,0,"[1Y_English 1, 1Y_English 1.1, 1Y_English 1.2]"
6,1Y_OOP,3,102,0,"[1Y_OOP, 1Y_OOP.1, 1Y_OOP.2]"
7,1Y_Introduction to Linux,3,81,0,"[1Y_Introduction to Linux, 1Y_Introduction to ..."
8,1Y_Linear Algebra,3,102,0,"[1Y_Linear Algebra, 1Y_Linear Algebra.1, 1Y_Li..."
9,1Y_Mathematical Analysis 1,3,102,0,"[1Y_Mathematical Analysis 1, 1Y_Mathematical A..."


Duplicate-column collapse report (dataset 2) — top 15:


,base_column,n_columns_collapsed,rows_with_any_value,rows_with_conflict,columns


Shapes before/after collapse:
df_pred_raw: (200, 115) -> (200, 77)
df_scores_raw: (275, 61) -> (275, 61)
Standardized columns:
df_pred cols: 77
df_scores cols: 61
Common columns (after standardization): 37
Columns only in df_pred: 40
Columns only in df_scores: 24
No header-collisions detected.
Common columns BEFORE standardization (raw): 18


,common_after_standardization_sample
0,1y average
1,1y average s1
2,1y average s2
3,1y cpsct
4,1y linear algebra
5,1y oop
6,2 programming
7,2 second
8,2y average
9,2y average s1


,df_pred_only_sample
0,1y digital systems
1,1y dsa1
2,1y english 1
3,1y english 2
4,1y foundational mathematics
5,1y information technology essentials
6,1y introduction to linux
7,1y introduction to statistics
8,1y mathematical analysis 1
9,2y computer architecture


,df_scores_only_sample
0,1y a com i
1,1y a com ii
2,1y algebra
3,1y analysis i
4,1y analysis ii
5,1y intensive english
6,1y intro prog c
7,1y proba stat
8,1y seminar
9,2y analysis iii


## 4) Understand the data: column meaning, semantic type, and encoding decision (for every column)

Goal:
- For every column we document:
  - **Meaning (inferred from name + samples)**
  - **Semantic type**: nominal / ordinal / numerical / multi-choice
  - **Recommended representation for ML** (no modeling here; we only decide how we *would* encode later)

Important note (defense-friendly):
- If the dataset does not provide a formal data dictionary, we infer meaning from column names and observed values.
- This is printed as a table so you can include it directly in your report.

In [257]:
def detect_multi_choice(series: pd.Series, sample_n: int = 200) -> bool:
    """Heuristic: multi-choice responses are often stored as a single string with separators."""
    if series.dtype != 'object':
        return False
    s = series.dropna().astype(str).head(sample_n)
    if s.empty:
        return False
    sep_hits = s.str.contains(r'[;,]|\|', regex=True).mean()
    return float(sep_hits) >= 0.15

def infer_semantic_type(col: str, series: pd.Series) -> tuple[str, str, str]:
    """Return (semantic_type, recommended_representation, inferred_meaning)."""
    name = col.lower()
    multi = detect_multi_choice(series)

    if multi:
        return (
            'multi-choice',
            'multi-hot (split then one-hot) or count features',
            'Multiple selections stored in one string',
        )

    # Year course grades / year averages
    if re.match(r'^[123]y_', name):
        return ('numerical', 'keep numeric (optionally scale later)', 'Course grade / year average')

    # Common survey fields
    if 'gender' in name:
        return ('nominal', 'one-hot', 'Gender category')
    if 'bac' in name or 'specialty' in name:
        return ('nominal', 'one-hot', 'BAC stream/specialty')
    if 'year of your studies' in name or 'currently enrolled' in name:
        return ('ordinal', 'ordinal encoding (First < Second < Third)', 'Academic year level')

    # Ordered bins (hours, sleep, stress, etc.)
    if 'hours per week' in name or 'how many hours' in name or 'hours do you sleep' in name:
        return ('ordinal', 'ordinal encoding (map bins to ordered integers)', 'Ordered time category')

    # Numeric rating scales
    if 'scale of 1 to 10' in name or 'rate your level' in name or name.strip().startswith(('1.', '2.', '3.', '4.')):
        return ('numerical', 'numeric (ensure range, then keep)', 'Numeric rating scale')

    # Yes/No style
    if series.dropna().isin(['Yes', 'No', 'yes', 'no', 'TRUE', 'FALSE', True, False]).mean() > 0.8:
        return ('nominal', 'binary (0/1)', 'Binary yes/no')

    # Default
    if series.dtype == 'object':
        return ('nominal', 'one-hot (or target encoding later)', 'Categorical/text response')
    return ('numerical', 'keep numeric', 'Numeric feature')

def column_dictionary(df: pd.DataFrame, sample_values: int = 5) -> pd.DataFrame:
    rows = []
    for col in df.columns:
        s = df[col]
        semantic_type, representation, meaning = infer_semantic_type(col, s)
        examples = s.dropna().astype(str).unique()[:sample_values].tolist()
        rows.append({
            'column': col,
            'pandas_dtype': str(s.dtype),
            'semantic_type': semantic_type,
            'recommended_representation': representation,
            'inferred_meaning': meaning,
            'non_null_pct': float(s.notna().mean()),
            'n_unique': int(s.nunique(dropna=True)),
            'example_values': examples,
        })
    return pd.DataFrame(rows)

dict_pred = column_dictionary(df_pred)
dict_scores = column_dictionary(df_scores)

print('Column dictionary — df_pred (all columns):')
display(dict_pred)
print('Column dictionary — df_scores (all columns):')
display(dict_scores)

Column dictionary — df_pred (all columns):


,column,pandas_dtype,semantic_type,recommended_representation,inferred_meaning,non_null_pct,n_unique,example_values
0,timestamp,object,nominal,one-hot (or target encoding later),Categorical/text response,1.0000,198,"[2025/01/08 6:00:12 PM GMT+1, 2025/01/08 6:00:..."
1,what is your gender,object,nominal,one-hot,Gender category,1.0000,2,"[Female, Male]"
2,in which year of your studies are you currentl...,object,ordinal,ordinal encoding (First < Second < Third),Academic year level,1.0000,4,"[First year, Fourth year, Second year, Thirs y..."
3,what was your bac specialty,object,nominal,one-hot,BAC stream/specialty,1.0000,3,"[Science, Mathematics, Technical maths]"
4,how many hours per week do you spend studying ...,object,ordinal,ordinal encoding (map bins to ordered integers),Ordered time category,1.0000,5,"[18-25, 10-17, 26-33, More than 41, 34-41]"
5,where do you prefer to study,object,nominal,one-hot (or target encoding later),Categorical/text response,1.0000,4,"[In your room (dorms), In the school's library..."
6,you prefer to study,object,nominal,one-hot (or target encoding later),Categorical/text response,1.0000,3,"[Alone, In peer, With a group]"
7,what learning methods do help you understand b...,object,multi-choice,multi-hot (split then one-hot) or count features,Multiple selections stored in one string,1.0000,40,"[Learning by practicing, Lectures;Learning by ..."
8,what types of resources do you use most,object,multi-choice,multi-hot (split then one-hot) or count features,Multiple selections stored in one string,1.0000,30,"[Online tutorials, Online documents;YouTube vi..."
9,do you use a planer or a schedule in your revi...,object,nominal,binary (0/1),Binary yes/no,1.0000,2,"[Yes, No]"


Column dictionary — df_scores (all columns):


,column,pandas_dtype,semantic_type,recommended_representation,inferred_meaning,non_null_pct,n_unique,example_values
0,what is your gender,object,nominal,one-hot,Gender category,1.0000,2,"[Male, Female]"
1,in which year of your studies are you currentl...,object,ordinal,ordinal encoding (First < Second < Third),Academic year level,1.0000,3,"[First year, Third year, Second year]"
2,what was your bac specialty,object,nominal,one-hot,BAC stream/specialty,1.0000,3,"[Mathematics, Science, Technical maths]"
3,how many hours per week do you spend studying ...,object,ordinal,ordinal encoding (map bins to ordered integers),Ordered time category,1.0000,5,"[18-25, 10-17, 34-41, More than 41 hours, 26-33]"
4,where do you prefer to study,object,nominal,one-hot (or target encoding later),Categorical/text response,1.0000,4,"[In your room (dorms), At home, In the school'..."
5,you prefer to study,object,nominal,one-hot (or target encoding later),Categorical/text response,1.0000,3,"[Alone, In peer, With a group]"
6,what learning methods do help you understand b...,object,multi-choice,multi-hot (split then one-hot) or count features,Multiple selections stored in one string,1.0000,42,"[Lectures, Learning by practicing, Project-bas..."
7,what types of resources do you use most,object,multi-choice,multi-hot (split then one-hot) or count features,Multiple selections stored in one string,0.9964,30,"[Online tutorials, Online documents, YouTube v..."
8,do you use a planer or a schedule in your revi...,object,nominal,binary (0/1),Binary yes/no,1.0000,2,"[No, Yes]"
9,for the next 8 questions you will have a list ...,object,nominal,one-hot (or target encoding later),Categorical/text response,1.0000,8,"[Pressure, Teachers' teaching method, Private ..."


## 5) Merge the two datasets (FULL OUTER JOIN on shared columns) + validate

Why the merge is needed:
- The information is split across two exports, so we need one unified table for analysis.
- We must keep **all rows** from both files and include **all columns** (shared + extra).

Shared vs extra columns logic:
- **Shared columns** = columns that exist in *both* datasets. They represent the common “schema”.
- **Extra columns** = columns that exist in only one dataset. They must be kept (they will be `NaN` for rows coming from the other dataset).

Merge strategy (as you requested):
- We perform a **full outer join** on the shared columns, meaning:
  - If a row matches across both files on the shared-column values, it becomes **one merged row** with extra columns filled.
  - If a row exists in only one file (no match), it is still kept in the output.
- Missing values are represented as `NaN` when a dataset does not have a particular extra column or when values are missing.

Validation performed right after merge:
- Confirm shared vs extra column counts
- Report merge result breakdown (matched rows vs left-only vs right-only)
- Ensure no duplicated column names
- Drop exact duplicate rows (defensible)
- Export the merged dataset **before any cleaning**

In [258]:
# -------------------------
# 5) MERGE (FULL OUTER JOIN ON SHARED COLUMNS)
# -------------------------

# Identify shared vs extra columns
shared_cols = sorted(set(df_pred.columns) & set(df_scores.columns))
pred_only = sorted(set(df_pred.columns) - set(df_scores.columns))
scores_only = sorted(set(df_scores.columns) - set(df_pred.columns))

print('Shared columns:', len(shared_cols))
print('Columns only in dataset 1 (df_pred):', len(pred_only))
print('Columns only in dataset 2 (df_scores):', len(scores_only))
display(pd.DataFrame({
    'metric': ['shared_columns', 'df_pred_only_columns', 'df_scores_only_columns'],
    'count': [len(shared_cols), len(pred_only), len(scores_only)],
}))

if not shared_cols:
    raise ValueError('No shared columns detected; cannot outer-join by shared columns.')

# IMPORTANT: pandas merge requires the join-key dtypes to match between left/right.
# Some shared columns (e.g., averages) may be numeric in one file and text in the other.
# We harmonize dtypes per shared column before merging.
def normalize_text_series(s: pd.Series) -> pd.Series:
    s = s.astype('string')
    s = s.str.strip()
    s = s.str.replace(r'\s+', ' ', regex=True)
    return s

def harmonize_shared_key_types(df_left: pd.DataFrame, df_right: pd.DataFrame, cols: list[str]) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    left = df_left.copy()
    right = df_right.copy()
    report = []
    for c in cols:
        l = left[c]
        r = right[c]
        l_dtype = str(l.dtype)
        r_dtype = str(r.dtype)

        # Parse-rate computed only among non-null values (so missingness doesn't hide parsability)
        l_nonnull = int(l.notna().sum())
        r_nonnull = int(r.notna().sum())
        l_num = pd.to_numeric(l, errors='coerce')
        r_num = pd.to_numeric(r, errors='coerce')
        l_rate = float(l_num.notna().sum() / l_nonnull) if l_nonnull else 0.0
        r_rate = float(r_num.notna().sum() / r_nonnull) if r_nonnull else 0.0

        # If both sides are mostly numeric-like, treat as numeric keys; else treat as text keys
        use_numeric = (l_rate >= 0.90 and r_rate >= 0.90 and (l_nonnull + r_nonnull) > 0)
        if use_numeric:
            left[c] = pd.to_numeric(l, errors='coerce')
            right[c] = pd.to_numeric(r, errors='coerce')
            chosen = 'numeric'
        else:
            left[c] = normalize_text_series(l)
            right[c] = normalize_text_series(r)
            chosen = 'text'

        report.append({
            'column': c,
            'left_dtype': l_dtype,
            'right_dtype': r_dtype,
            'left_numeric_parse_rate': l_rate,
            'right_numeric_parse_rate': r_rate,
            'chosen_key_type': chosen,
        })

    report_df = pd.DataFrame(report)
    print('Shared-key dtype harmonization summary (top 25 by dtype mismatch):')
    report_df['dtype_mismatch'] = report_df['left_dtype'] != report_df['right_dtype']
    display(report_df.sort_values(['dtype_mismatch','chosen_key_type','column'], ascending=[False, True, True]).head(25))
    return left, right, report_df

df_pred_m, df_scores_m, key_type_report = harmonize_shared_key_types(df_pred, df_scores, shared_cols)

# Check for duplicate keys (many-to-many joins can inflate row counts)
dup_pred_keys = int(df_pred_m.duplicated(subset=shared_cols, keep=False).sum())
dup_scores_keys = int(df_scores_m.duplicated(subset=shared_cols, keep=False).sum())
print('Rows with duplicated shared-key values in df_pred:', dup_pred_keys)
print('Rows with duplicated shared-key values in df_scores:', dup_scores_keys)

merge_validate = 'one_to_one' if (dup_pred_keys == 0 and dup_scores_keys == 0) else 'm:m'
print('Merge validation mode:', merge_validate)

# Full outer join on the shared columns (align shared columns by values)
df_merged = pd.merge(
    df_pred_m,
    df_scores_m,
    how='outer',
    on=shared_cols,
    sort=False,
    indicator=True,
    validate=merge_validate,
 )

print('Merged shape:', df_merged.shape)
assert df_merged.columns.is_unique, 'Duplicate column names detected after merge.'

print('Merge breakdown (_merge):')
display(df_merged['_merge'].value_counts().to_frame('count'))

# Drop the merge indicator (audit-only)
df_merged = df_merged.drop(columns=['_merge'])

# Ensure no exact duplicate rows
before = len(df_merged)
df_merged = df_merged.drop_duplicates(keep='first')
after = len(df_merged)
print('Exact duplicate rows removed:', before - after)

# Add a simple row identifier (traceability across outputs)
df_merged = df_merged.reset_index(drop=True)
df_merged.insert(0, 'record_id', np.arange(1, len(df_merged) + 1))
print('Added record_id. Unique:', df_merged['record_id'].is_unique)

# Save merged dataset BEFORE cleaning (requested artifact)
out_merged_raw = data_dir / 'ensia_merged_full_outer_before_cleaning.csv'
df_merged.to_csv(out_merged_raw, index=False)
print('Saved merged (before cleaning):', out_merged_raw)

# -------------------------
# 6) Detect issues (DO NOT CLEAN YET)
# -------------------------
issue_report = {}

# Missing values
missing_pct = df_merged.isna().mean().sort_values(ascending=False)
issue_report['missing_pct'] = missing_pct
print('Missingness (top 25):')
display(missing_pct.head(25).to_frame('missing_pct'))

# Exact duplicate rows (should be 0 after drop_duplicates)
dup_rows = int(df_merged.duplicated(keep='first').sum())
issue_report['exact_duplicate_rows'] = dup_rows
print('Exact duplicate rows (post-drop):', dup_rows)

# Inconsistent categories (whitespace/casing artifacts)
def normalize_text_value(x: object) -> object:
    if pd.isna(x):
        return x
    s = str(x)
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)
    return s

inconsistent = []
obj_cols = df_merged.select_dtypes(include=['object','string']).columns.tolist()
for col in obj_cols:
    s = df_merged[col]
    raw_unique = s.dropna().astype(str).nunique()
    norm_unique = s.map(normalize_text_value).dropna().astype(str).nunique()
    if norm_unique < raw_unique:
        inconsistent.append({'column': col, 'raw_unique': int(raw_unique), 'normalized_unique': int(norm_unique)})

inconsistent_df = (
    pd.DataFrame(inconsistent).sort_values(['raw_unique'], ascending=False)
    if inconsistent
    else pd.DataFrame(columns=['column', 'raw_unique', 'normalized_unique'])
 )
issue_report['inconsistent_categories'] = inconsistent_df
print('Potential inconsistent categories (top 20):')
display(inconsistent_df.head(20))

# Out-of-range indicators (conservative checks)
def to_numeric_coerce(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors='coerce')

def is_id_like(col_name: str) -> bool:
    n = str(col_name).lower().strip()
    return (n == 'record_id') or n.endswith('_id')

numeric_like = []
for col in df_merged.columns:
    if is_id_like(col):
        continue
    s = df_merged[col]
    if pd.api.types.is_numeric_dtype(s):
        numeric_like.append(col)
    elif str(s.dtype) in {'object', 'string'}:
        s_num = to_numeric_coerce(s)
        if float(s_num.notna().mean()) >= 0.60:
            numeric_like.append(col)

oor = []
for col in numeric_like:
    if is_id_like(col):
        continue
    s_num = to_numeric_coerce(df_merged[col])
    if int(s_num.notna().sum()) == 0:
        continue
    neg = int((s_num < 0).sum())
    gt100 = int((s_num > 100).sum())
    name = col.lower()
    scale_1_10 = ('scale of 1 to 10' in name) or ('rate your level' in name) or ('rate your satisfaction' in name)
    gt10 = int((s_num > 10).sum()) if scale_1_10 else 0
    lt1 = int((s_num < 1).sum()) if scale_1_10 else 0
    if neg or gt100 or gt10 or lt1:
        oor.append({
            'column': col,
            'negatives': neg,
            'gt_100': gt100,
            'scale_1_10_suspected': bool(scale_1_10),
            'lt_1': lt1,
            'gt_10': gt10,
        })

oor_df = (
    pd.DataFrame(oor).sort_values(['scale_1_10_suspected', 'gt_10', 'lt_1', 'gt_100', 'negatives'], ascending=False)
    if oor
    else pd.DataFrame(columns=['column','negatives','gt_100','scale_1_10_suspected','lt_1','gt_10'])
 )
issue_report['out_of_range'] = oor_df
print('Out-of-range indicators (top 20):')
display(oor_df.head(20))

print('Issue report keys:', list(issue_report.keys()))

Shared columns: 37
Columns only in dataset 1 (df_pred): 40
Columns only in dataset 2 (df_scores): 24


,metric,count
0,shared_columns,37
1,df_pred_only_columns,40
2,df_scores_only_columns,24


Shared-key dtype harmonization summary (top 25 by dtype mismatch):


,column,left_dtype,right_dtype,left_numeric_parse_rate,right_numeric_parse_rate,chosen_key_type,dtype_mismatch
9,2y average s1,float64,object,1.0000,0.0000,text,True
0,1y average,float64,float64,1.0000,1.0000,numeric,False
1,1y average s1,float64,float64,1.0000,1.0000,numeric,False
2,1y average s2,float64,float64,1.0000,1.0000,numeric,False
6,2 programming,int64,int64,1.0000,1.0000,numeric,False
8,2y average,float64,float64,1.0000,1.0000,numeric,False
10,2y average s2,float64,float64,1.0000,1.0000,numeric,False
13,3 computer science,int64,int64,1.0000,1.0000,numeric,False
15,4 mathematics,int64,int64,1.0000,1.0000,numeric,False
25,how much do you like the field of artificial i...,int64,int64,1.0000,1.0000,numeric,False


Rows with duplicated shared-key values in df_pred: 0
Rows with duplicated shared-key values in df_scores: 0
Merge validation mode: one_to_one
Merged shape: (475, 102)
Merge breakdown (_merge):


...

Exact duplicate rows (post-drop): 0
Potential inconsistent categories (top 20):


,column,raw_unique,normalized_unique


Out-of-range indicators (top 20):


,column,negatives,gt_100,scale_1_10_suspected,lt_1,gt_10


Issue report keys: ['missing_pct', 'exact_duplicate_rows', 'inconsistent_categories', 'out_of_range']


In [ ]:
# -------------------------

# Folder hygiene: move merged artifact into data/intermediate

# -------------------------



merged_name = 'ensia_merged_full_outer_before_cleaning.csv'

src_merged = data_dir / merged_name

dst_merged = intermediate_dir / merged_name



if src_merged.exists():

    if dst_merged.exists():

        dst_merged.unlink()

    src_merged.rename(dst_merged)

    print('Moved merged artifact to:', dst_merged)

else:

    print('Merged artifact not found at root (ok):', src_merged)


## 5a) Merged dataset summary (report-ready stats)

This section provides a compact statistical overview of the merged dataset **right after the FULL OUTER JOIN** and **before any cleaning/splitting**, so you can use it directly in your methodology/report.

In [259]:
# Merged dataset summary (after FULL OUTER JOIN, before cleaning/splitting)
df = df_merged.copy()

print('Merged dataset:')
print(' - rows:', len(df))
print(' - cols:', df.shape[1])

# Column type breakdown
n_num = int(df.select_dtypes(include=['number']).shape[1])
n_cat = int(df.select_dtypes(include=['object','string','bool','category']).shape[1])
n_other = int(df.shape[1] - n_num - n_cat)
display(pd.DataFrame([
    {'group': 'numeric', 'n_cols': n_num},
    {'group': 'categorical_like', 'n_cols': n_cat},
    {'group': 'other', 'n_cols': n_other},
]))

# Basic ID sanity (if present)
if 'record_id' in df.columns:
    print('record_id unique:', bool(df['record_id'].is_unique))
    print('record_id missing:', int(df['record_id'].isna().sum()))

# Missingness summary distribution
missing_pct = df.isna().mean()
summary_rows = []
for thr in [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]:
    if thr == 0.0:
        n = int((missing_pct == 0.0).sum())
        label = '0% missing (complete)'
    elif thr == 1.0:
        n = int((missing_pct == 1.0).sum())
        label = '100% missing (empty)'
    else:
        n = int((missing_pct >= thr).sum())
        label = f'>= {int(thr*100)}% missing'
    summary_rows.append({'metric': label, 'n_cols': n, 'pct_of_cols': float(n/df.shape[1]) if df.shape[1] else float('nan')})
display(pd.DataFrame(summary_rows))

# Top missing columns (limited for readability)
top_missing = missing_pct.sort_values(ascending=False).head(25).to_frame('missing_pct')
print('Top 25 columns by missingness:')
display(top_missing)

# Fully-empty columns (should generally be none after later cleaning)
empty_cols = missing_pct[missing_pct == 1.0].index.tolist()
print('100% empty columns count:', len(empty_cols))
if empty_cols:
    display(pd.DataFrame({'empty_column': empty_cols}).head(50))

# Approximate merge-origin breakdown using presence/absence of side-specific columns
if 'pred_only' in globals() and 'scores_only' in globals():
    pred_only_cols = [c for c in pred_only if c in df.columns]
    scores_only_cols = [c for c in scores_only if c in df.columns]

    left_only_mask = df[scores_only_cols].isna().all(axis=1) if scores_only_cols else pd.Series(False, index=df.index)
    right_only_mask = df[pred_only_cols].isna().all(axis=1) if pred_only_cols else pd.Series(False, index=df.index)
    both_mask = ~(left_only_mask | right_only_mask)
    origin_df = pd.DataFrame([
        {'origin': 'likely df_pred-only row', 'count': int(left_only_mask.sum())},
        {'origin': 'likely df_scores-only row', 'count': int(right_only_mask.sum())},
        {'origin': 'likely matched row (both sides)', 'count': int(both_mask.sum())},
    ])
    origin_df['pct'] = origin_df['count'] / len(df) if len(df) else float('nan')
    print('Approximate origin breakdown (based on side-specific columns being all-NaN):')
    display(origin_df)

# Year distribution if we can detect the year/enrollment column
year_candidates = [c for c in df.columns if ('currently enrolled' in str(c).lower()) or ('year of your studies' in str(c).lower())]
if year_candidates:
    year_col = year_candidates[0]
    print('Detected year/enrollment column candidate:', year_col)
    vc = df[year_col].value_counts(dropna=False).head(25)
    display(vc.to_frame('count'))
else:
    print('No year/enrollment column detected at this stage.')

Merged dataset:
 - rows: 475
 - cols: 102


,group,n_cols
0,numeric,17
1,categorical_like,85
2,other,0


record_id unique: True
record_id missing: 0


,metric,n_cols,pct_of_cols
0,0% missing (complete),26,0.2549
1,>= 10% missing,75,0.7353
2,>= 25% missing,75,0.7353
3,>= 50% missing,73,0.7157
4,>= 75% missing,64,0.6275
5,>= 90% missing,27,0.2647
6,100% missing (empty),0,0.0000


Top 25 columns by missingness:


,missing_pct
3y entrepreneurship and innovation,0.9916
3y machine learning,0.9916
3y data mining,0.9916
3y operations research,0.9916
3y stochastic,0.9916
3y software engineering,0.9916
3y networks and protocols,0.9916
3y mobile development,0.9916
3y advanced databases,0.9916
3y computer and network security,0.9916


100% empty columns count: 0
Approximate origin breakdown (based on side-specific columns being all-NaN):


,origin,count,pct
0,likely df_pred-only row,200,0.4211
1,likely df_scores-only row,275,0.5789
2,likely matched row (both sides),0,0.0000


Detected year/enrollment column candidate: in which year of your studies are you currently enrolled


,count
in which year of your studies are you currently enrolled,
First year,215
Second year,164
Third year,48
Thirs year,33
Fourth year,15


## 7) Preprocessing TODO list (derived from the Issue Report) + start cleaning

We now translate the detected problems into a concrete TODO list, then start fixing them step-by-step.

### TODO list
- [ ] Remove exact duplicate rows (safe and defensible).
- [ ] Normalize text/categorical values (trim whitespace, collapse repeated spaces, harmonize obvious labels).
- [ ] Convert numeric-like columns stored as text into numeric (invalid values become missing).
- [ ] Handle out-of-range values conservatively (set invalid values to missing).
- [ ] Impute remaining missing values (numeric → median, categorical → `'Unknown'`).

The next cells implement these TODO items one by one (Markdown justification, then code).

In [260]:
df_clean = df_merged.copy()
before = len(df_clean)
df_clean = df_clean.drop_duplicates(keep='first')
after = len(df_clean)
print('Rows before:', before)
print('Rows after removing exact duplicates:', after)
print('Removed:', before - after)

Rows before: 475
Rows after removing exact duplicates: 475
Removed: 0


## 9) Cleaning step 2 — normalize categorical/text values

Why this is necessary:
- In surveys, the same category can appear with trailing spaces or inconsistent formatting (e.g., `Male`, `male `).
- This creates *artificial new categories*, which harms analysis and encoding.

Cleaning logic:
- For all text columns: trim whitespace and collapse repeated spaces.
- We do **not** aggressively change meaning (no forced lowercasing everywhere).

In [261]:
obj_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
for col in obj_cols:
    df_clean[col] = df_clean[col].map(lambda x: normalize_text_value(x))

# Optional small harmonizations for obvious fields (gender / yes-no)
for col in df_clean.columns:
    name = col.lower()
    if 'gender' in name and col in df_clean.columns:
        df_clean[col] = df_clean[col].replace({'male': 'Male', 'female': 'Female'})

print('Done normalizing text columns.')

Done normalizing text columns.


## 10) Cleaning step 3 — convert numeric-like columns to numeric

Why this is necessary:
- Many grade/rating columns are read as text because of missing values or formatting.
- Numerical representation is required for statistics, outlier checks, feature selection, and later ML.

Cleaning logic:
- Detect numeric-like columns based on parse rate.
- Convert using `pd.to_numeric(..., errors='coerce')` (invalid values become NaN).

In [262]:
def numeric_parse_rate(series: pd.Series) -> float:
    """Return numeric parse success rate among non-missing values."""
    s_non = series.dropna()
    if s_non.empty:
        return 0.0
    s_num = pd.to_numeric(s_non.astype(str).str.replace(',', '.', regex=False), errors='coerce')
    return float(s_num.notna().mean())

def is_year_prefixed_col(col: str) -> bool:
    # Matches year-prefixed academic columns like "1y ...", "2y_...", ..., "4y ..."
    return bool(re.match(r'^[1234]y(?:\s|_|$)', str(col).lower()))

def should_force_year_col_to_numeric(series: pd.Series) -> bool:
    """Force to numeric only if values are truly numeric-like (not categories like High/Low)."""
    s_non = series.dropna()
    if s_non.empty:
        return False
    s_txt = s_non.astype(str).str.strip()
    # If many values contain letters, treat as categorical (e.g., High/Low/Very high)
    alpha_rate = float(s_txt.str.contains(r"[A-Za-z]", regex=True).mean())
    if alpha_rate >= 0.10:
        return False
    s_num = pd.to_numeric(s_txt.str.replace(',', '.', regex=False), errors='coerce')
    return float(s_num.notna().mean()) >= 0.90

numeric_candidates = []
forced_year_numeric = []
skipped_year_categorical = []

for col in df_clean.columns:
    s = df_clean[col]
    if pd.api.types.is_numeric_dtype(s):
        continue

    if is_year_prefixed_col(col):
        # Only force numeric if it really looks numeric among non-null values
        if should_force_year_col_to_numeric(s):
            forced_year_numeric.append(col)
            df_clean[col] = pd.to_numeric(s.astype(str).str.replace(',', '.', regex=False), errors='coerce')
        else:
            skipped_year_categorical.append(col)
        continue

    rate = numeric_parse_rate(s)
    if rate >= 0.60:
        numeric_candidates.append({'column': col, 'parse_rate': rate})

numeric_candidates_df = (
    pd.DataFrame(numeric_candidates).sort_values('parse_rate', ascending=False)
    if numeric_candidates
    else pd.DataFrame(columns=['column','parse_rate'])
 )
print('Forced year-prefixed columns to numeric:', len(forced_year_numeric))
display(pd.DataFrame({'forced_year_numeric_cols_sample': forced_year_numeric[:30]}))
print('Skipped year-prefixed columns (categorical-like):', len(skipped_year_categorical))
display(pd.DataFrame({'skipped_year_categorical_cols_sample': skipped_year_categorical[:30]}))

print('Numeric-like candidates (top 30):')
display(numeric_candidates_df.head(30))

for col in numeric_candidates_df['column'].tolist():
    df_clean[col] = pd.to_numeric(df_clean[col].astype(str).str.replace(',', '.', regex=False), errors='coerce')

print('Converted numeric-like columns to numeric where possible.')

Forced year-prefixed columns to numeric: 0


,forced_year_numeric_cols_sample


Skipped year-prefixed columns (categorical-like): 61


,skipped_year_categorical_cols_sample
0,2y average s1
1,1y foundational mathematics
2,1y dsa1
3,1y digital systems
4,1y information technology essentials
5,1y english 1
6,1y cpsct
7,1y oop
8,1y introduction to linux
9,1y linear algebra


Numeric-like candidates (top 30):


,column,parse_rate


Converted numeric-like columns to numeric where possible.


## 11) Cleaning step 4 — fix out-of-range values (defensible rules)

Why this is necessary:
- Out-of-range values usually come from data-entry mistakes or inconsistent scales.
- Keeping them can distort distributions and feature selection.

Cleaning logic (conservative):
- For suspected 1–10 rating columns, set values outside [1, 10] to `NaN`.
- For any numeric column, set negative values to `NaN` (grades/ratings should not be negative).

We do **not** hard-code a grading scale (0–20 vs 0–100) unless explicitly documented.

In [263]:
num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
fixed_counts = []

def is_id_like(col_name: str) -> bool:
    n = str(col_name).lower().strip()
    return (n == 'record_id') or n.endswith('_id')

for col in num_cols:
    if is_id_like(col):
        continue
    s = df_clean[col]
    before_na = int(s.isna().sum())
    name = col.lower()
    scale_1_10 = ('scale of 1 to 10' in name) or ('rate your level' in name) or ('rate your satisfaction' in name)

    # negatives => NaN
    df_clean.loc[df_clean[col] < 0, col] = np.nan

    if scale_1_10:
        df_clean.loc[(df_clean[col] < 1) | (df_clean[col] > 10), col] = np.nan

    after_na = int(df_clean[col].isna().sum())
    if after_na > before_na:
        fixed_counts.append({'column': col, 'new_nans_added': after_na - before_na, 'scale_1_10': bool(scale_1_10)})

fixed_df = pd.DataFrame(fixed_counts).sort_values('new_nans_added', ascending=False) if fixed_counts else pd.DataFrame(columns=['column','new_nans_added','scale_1_10'])
print('Out-of-range fixes applied (top 20):')
display(fixed_df.head(20))

Out-of-range fixes applied (top 20):


,column,new_nans_added,scale_1_10


## 12) Cleaning step 5 — impute missing values (optional / configurable)

Why this step exists:
- Many ML/data mining algorithms require a complete matrix (no NaNs).
- Imputation must be justified per type:
  - **Numeric (survey/ratings)**: median (robust to outliers)
  - **Categorical**: explicit `'Unknown'` category (keeps the information that it was missing)

Important note about grades/modules (very important for your report):
- Imputing **academic grades** can create “fake grades”.
- So by default we **do NOT impute year grade/module columns** (we keep them as missing).

You can control this behavior using the flags in the next code cell.

In [264]:
def is_year_grade_col(col: str) -> bool:
    return bool(re.match(r'^[1234]y(?:\s|_|$)', str(col).lower()))

# -------------------------
# Year-aware missing-value handling (recommended)
# -------------------------
# Key idea: split-by-year FIRST (conceptually), then impute ONLY survey features within each year group.
# We avoid fabricating grades and avoid "structural missingness" causing false imputation.

ENABLE_IMPUTATION = True
IMPUTE_YEAR_GRADES = False

# Detect + normalize year column early (needed for groupwise imputation)
year_col = find_year_column(df_clean.columns.tolist())
df_clean[year_col] = df_clean[year_col].map(normalize_year_value)

# Optional helper (useful in later steps / debugging)
year_map = {'First year': 1, 'Second year': 2, 'Third year': 3, 'Fourth year': 4}
df_clean['year_ordinal'] = df_clean[year_col].map(year_map)

year_grade_cols = [c for c in df_clean.columns if is_year_grade_col(c)]
non_grade_cols = [c for c in df_clean.columns if c not in year_grade_cols]

# -------------------------
# Diagnostics: year-label vs year-prefixed grades presence
# This helps explain cases like "1y english" being empty in 1Y split but present elsewhere in full data.
# -------------------------
def any_nonempty(df: pd.DataFrame, cols: list[str]) -> pd.Series:
    if not cols:
        return pd.Series(False, index=df.index)
    sub = df[cols]
    # categorical-aware: treat blank strings as empty
    out = pd.Series(False, index=df.index)
    for c in cols:
        s = sub[c]
        if pd.api.types.is_numeric_dtype(s):
            out = out | s.notna()
        else:
            st = s.astype('string')
            out = out | (st.notna() & (st.str.strip() != ''))
    return out

y1_cols = [c for c in year_grade_cols if re.match(r'^1y(?:\s|_|$)', str(c).lower())]
y2_cols = [c for c in year_grade_cols if re.match(r'^2y(?:\s|_|$)', str(c).lower())]
y3_cols = [c for c in year_grade_cols if re.match(r'^3y(?:\s|_|$)', str(c).lower())]
y4_cols = [c for c in year_grade_cols if re.match(r'^4y(?:\s|_|$)', str(c).lower())]

has_1y = any_nonempty(df_clean, y1_cols)
has_2y = any_nonempty(df_clean, y2_cols)
has_3y = any_nonempty(df_clean, y3_cols)
has_4y = any_nonempty(df_clean, y4_cols)

inferred_year = pd.Series(np.nan, index=df_clean.index, dtype='float')
inferred_year.loc[has_1y] = 1
inferred_year.loc[has_2y] = 2
inferred_year.loc[has_3y] = 3
inferred_year.loc[has_4y] = 4
df_clean['inferred_year_from_grades'] = inferred_year

mismatch_mask = df_clean['year_ordinal'].notna() & df_clean['inferred_year_from_grades'].notna() & (df_clean['year_ordinal'] != df_clean['inferred_year_from_grades'])
print('Year label vs inferred-from-grades mismatches (count):', int(mismatch_mask.sum()))
if int(mismatch_mask.sum()) > 0:
    display(
        df_clean.loc[mismatch_mask, [year_col, 'year_ordinal', 'inferred_year_from_grades']].head(25)
    )

# -------------------------
# Imputation (year-aware, non-grade columns only)
# -------------------------
num_cols = df_clean[non_grade_cols].select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in non_grade_cols if c not in num_cols]

if ENABLE_IMPUTATION:
    # 1) Numeric: fill NaNs with median within the same declared year (fallback to global median)
    for col in num_cols:
        if df_clean[col].isna().any():
            per_year_med = df_clean.groupby(year_col, dropna=False)[col].transform('median')
            df_clean[col] = df_clean[col].fillna(per_year_med)
            # global fallback (if a whole year-group is all-NaN)
            df_clean[col] = df_clean[col].fillna(df_clean[col].median())

    # 2) Categorical/text: fill NaNs/blanks with mode within the same declared year (fallback 'Unknown')
    def mode_or_unknown(s: pd.Series) -> object:
        s2 = s.astype('string')
        s2 = s2[~(s2.isna() | (s2.str.strip() == ''))]
        if s2.empty:
            return 'Unknown'
        m = s2.mode(dropna=True)
        return m.iloc[0] if not m.empty else 'Unknown'

    # Compute per-year fill values once per column (fast enough for this dataset size)
    for col in cat_cols:
        s = df_clean[col]
        st = s.astype('string')
        missing_mask = st.isna() | (st.str.strip() == '')
        if not bool(missing_mask.any()):
            continue
        fill_by_year = df_clean.groupby(year_col, dropna=False)[col].apply(mode_or_unknown)
        # Map each row's year to its fill value
        df_clean.loc[missing_mask, col] = df_clean.loc[missing_mask, year_col].map(fill_by_year).fillna('Unknown')

    # 3) Grades (year-prefixed): by default DO NOT impute (avoid fabricating grades)
    if IMPUTE_YEAR_GRADES:
        numeric_grade_cols = [c for c in year_grade_cols if pd.api.types.is_numeric_dtype(df_clean[c])]
        for col in numeric_grade_cols:
            if df_clean[col].isna().any():
                per_year_med = df_clean.groupby(year_col, dropna=False)[col].transform('median')
                df_clean[col] = df_clean[col].fillna(per_year_med)

    print('Imputation enabled (year-aware).')
    print('Non-grade numeric imputed (per-year median):', len(num_cols))
    print('Non-grade categorical imputed (per-year mode):', len(cat_cols))
    print('Year grade/module columns detected (not imputed):', len(year_grade_cols))
    print('Median-impute year grades:', IMPUTE_YEAR_GRADES)
else:
    print('Imputation skipped (ENABLE_IMPUTATION=False). df_clean keeps NaN values.')
    print('Year grade/module columns detected:', len(year_grade_cols))

print('Missingness after this step (top 15):')
display(df_clean.isna().mean().sort_values(ascending=False).head(15).to_frame('missing_pct'))
print('df_clean shape:', df_clean.shape)

Year label vs inferred-from-grades mismatches (count): 246


,in which year of your studies are you currently enrolled,year_ordinal,inferred_year_from_grades
0,Second year,2,1.0000
1,Second year,2,1.0000
2,Second year,2,1.0000
4,Second year,2,1.0000
5,Third year,3,2.0000
6,Second year,2,1.0000
7,Second year,2,1.0000
8,Second year,2,1.0000
9,Second year,2,1.0000
10,Second year,2,1.0000


Imputation enabled (year-aware).
Non-grade numeric imputed (per-year median): 10
Non-grade categorical imputed (per-year mode): 24
Year grade/module columns detected (not imputed): 69
Median-impute year grades: False
Missingness after this step (top 15):


,missing_pct
3y entrepreneurship and innovation,0.9916
3y machine learning,0.9916
3y stochastic,0.9916
3y software engineering,0.9916
3y data mining,0.9916
3y operations research,0.9916
3y mobile development,0.9916
3y networks and protocols,0.9916
3y advanced databases,0.9916
3y computer and network security,0.9916


df_clean shape: (475, 104)


## 12a) Missingness report — empty values per column

We report **how many empty values** each column has after cleaning/imputation.

Definition of “empty” depends on the column type:
- **Numeric columns**: empty = `NaN`
- **Categorical/text columns**: empty = `NaN` **or** blank/whitespace-only string (`""`, `"   "`)

In [265]:
# Empty values per column (numeric vs categorical-aware)
n_rows = len(df_clean)
rows = []
for col in df_clean.columns:
    s = df_clean[col]
    is_num = pd.api.types.is_numeric_dtype(s)
    if is_num:
        empty_mask = s.isna()
        kind = 'numeric'
    else:
        s_txt = s.astype('string')
        empty_mask = s_txt.isna() | (s_txt.str.strip() == '')
        kind = 'categorical'
    empty_count = int(empty_mask.sum())
    rows.append({
        'column': col,
        'kind': kind,
        'dtype': str(s.dtype),
        'empty_count': empty_count,
        'empty_pct': (empty_count / n_rows) if n_rows else float('nan'),
        'non_empty_count': int(n_rows - empty_count),
    })

missingness_report = (
    pd.DataFrame(rows)
    .sort_values(['empty_pct', 'empty_count', 'column'], ascending=[False, False, True])
    .reset_index(drop=True)
 )
print('Empty values per column (ALL columns, sorted by empty_pct DESC):')
display(missingness_report)
print('Full report rows:', len(missingness_report))

Empty values per column (ALL columns, sorted by empty_pct DESC):


,column,kind,dtype,empty_count,empty_pct,non_empty_count
0,3y advanced databases,categorical,object,471,0.9916,4
1,3y computer and network security,categorical,object,471,0.9916,4
2,3y data mining,categorical,object,471,0.9916,4
3,3y entrepreneurship and innovation,categorical,object,471,0.9916,4
4,3y machine learning,categorical,object,471,0.9916,4
5,3y mobile development,categorical,object,471,0.9916,4
6,3y networks and protocols,categorical,object,471,0.9916,4
7,3y numerical methods and optimisation,categorical,object,471,0.9916,4
8,3y operations research,categorical,object,471,0.9916,4
9,3y software engineering,categorical,object,471,0.9916,4


Full report rows: 104


## 12b) Drop 100% empty columns + timestamp column

Why this step is necessary:
- Some exports include metadata columns (e.g., `timestamp`) that are not useful for modeling.
- Fully-empty columns (100% missing) add noise, break some algorithms, and waste memory.

Rule:
- Drop any column named like `timestamp` (case-insensitive).
- Drop any column where **all** values are missing.
- Print a report of what was removed (for your methodology/report).

In [266]:
# Drop timestamp + fully-empty columns (100% missing)
timestamp_cols = [c for c in df_clean.columns if re.search(r'time\s*stamp|timestamp', str(c), flags=re.IGNORECASE)]
empty_cols = [c for c in df_clean.columns if df_clean[c].isna().all()]

to_drop = sorted(set(timestamp_cols + empty_cols))
drop_report = []
for c in to_drop:
    reason = []
    if c in timestamp_cols:
        reason.append('timestamp')
    if c in empty_cols:
        reason.append('100% empty')
    drop_report.append({'column': c, 'reason': ' + '.join(reason)})

print('Columns to drop:', len(to_drop))
if drop_report:
    display(pd.DataFrame(drop_report).sort_values(['reason','column']))

if to_drop:
    df_clean = df_clean.drop(columns=to_drop)
    print('df_clean shape after dropping:', df_clean.shape)
else:
    print('No columns dropped.')

Columns to drop: 1


,column,reason
0,timestamp,timestamp


df_clean shape after dropping: (475, 103)


## 13) Save full cleaned dataset (do not overwrite raw files)

Why this is necessary:
- The cleaned dataset is the main artifact used by later steps (year splits, feature selection).
- Saving as CSV makes the pipeline reproducible and easy to share.

Output file:
- `ensia_full_cleaned.csv`

In [267]:
# -------------------------
# 13) Save outputs
#   - Cleaned (analysis-friendly): ensia_full_cleaned.csv
#   - Model-ready (encoded):      ensia_full_model_ready.csv
# -------------------------

def normalize_year_value(x: object) -> object:
    """Normalize academic year labels + fix typos like 'Thirs year' -> 'Third year'."""
    if pd.isna(x):
        return x
    s = str(x).strip()
    s = re.sub(r'\s+', ' ', s)
    s_low = s.lower()

    # Common variants / typos
    if 'thirs' in s_low:
        s = 'Third year'
        s_low = 'third year'

    # Numeric-ish variants
    if re.search(r'\b1(st)?\b', s_low) or 'first' in s_low:
        return 'First year'
    if re.search(r'\b2(nd)?\b', s_low) or 'second' in s_low:
        return 'Second year'
    if re.search(r'\b3(rd)?\b', s_low) or 'third' in s_low:
        return 'Third year'
    if re.search(r'\b4(th)?\b', s_low) or 'fourth' in s_low:
        return 'Fourth year'

    # If unknown, keep normalized original (but trimmed)
    return s


def find_year_column(columns: list[str]) -> str:
    candidates = [c for c in columns if 'year of your studies' in c.lower() or 'currently enrolled' in c.lower()]
    if not candidates:
        raise ValueError('Could not find a year/enrollment column. Please set it manually.')
    return candidates[0]


year_col = find_year_column(df_clean.columns.tolist())
print('Detected year column:', year_col)

# Fix year label typos / normalize categories BEFORE saving/splitting
df_clean[year_col] = df_clean[year_col].map(normalize_year_value)

# -------------------------
# Build model-ready dataframe (ordinal + multi-choice handling + numeric bins + obstacles)
# -------------------------
df_model = df_clean.copy()


def detect_multi_choice(series: pd.Series, sample_n: int = 200) -> bool:
    """Heuristic: multi-choice responses are often stored as a single string with separators."""
    if series.dtype != 'object' and str(series.dtype) != 'string':
        return False
    s = series.dropna().astype(str).head(sample_n)
    if s.empty:
        return False
    sep_hits = s.str.contains(r'[;,]|\|', regex=True).mean()
    return float(sep_hits) >= 0.15


def token_to_safe_feature(token: str) -> str:
    t = str(token).strip().lower()
    t = re.sub(r'\s+', '_', t)
    t = re.sub(r'[^a-z0-9_]+', '', t)
    return t[:80] if len(t) > 80 else t


def make_unique_cols(cols: list[str]) -> list[str]:
    seen: dict[str, int] = {}
    out: list[str] = []
    for c in cols:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}__dup{seen[c]}")
    return out


def multi_hot_expand(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """Expand a multi-choice text column into multi-hot indicator columns (row-wise OR)."""
    s = df[column]
    tokens = (
        s.astype('string')
        .fillna('')
        .str.split(r'[;,\|]', regex=True)
    )
    exploded = tokens.explode().astype('string').str.strip()
    exploded = exploded.replace('', pd.NA).dropna()
    if exploded.empty:
        return df

    unique_tokens = int(exploded.nunique(dropna=True))
    if unique_tokens > 250:
        print(f"WARNING: '{column}' expands to {unique_tokens} unique tokens (wide matrix).")

    dummies = pd.get_dummies(exploded, prefix=column, prefix_sep='__')
    safe_cols = make_unique_cols([token_to_safe_feature(c) for c in dummies.columns])
    dummies.columns = safe_cols
    dummies = dummies.groupby(level=0).max()

    out = df.drop(columns=[column]).join(dummies, how='left')
    out[safe_cols] = out[safe_cols].fillna(0).astype('int8')
    return out


# 1) Multi-choice columns -> multi-hot
multi_choice_cols = []
for col in df_model.columns:
    if col == 'record_id':
        continue
    if detect_multi_choice(df_model[col]):
        multi_choice_cols.append(col)

print('Detected multi-choice columns:', len(multi_choice_cols))
if multi_choice_cols:
    display(pd.DataFrame({'multi_choice_cols_sample': multi_choice_cols[:30]}))

for col in multi_choice_cols:
    df_model = multi_hot_expand(df_model, col)

# 2) Ordinal encoding: academic year
year_map = {'First year': 1, 'Second year': 2, 'Third year': 3, 'Fourth year': 4}
df_model['year_ordinal'] = df_model[year_col].map(year_map).astype('float')

# 2b) Ordinal encoding: module-level Likert values (Very low .. Very high)
# Common for module difficulty/effort/etc.
def normalize_ordinal_token_series(s: pd.Series) -> pd.Series:
    t = s.astype('string').str.strip().str.lower()
    t = t.str.replace('_', ' ', regex=False).str.replace('-', ' ', regex=False)
    t = t.str.replace(r'\s+', ' ', regex=True)
    # normalize common variants
    t = t.str.replace(r'\bvery\s*high\b', 'very high', regex=True)
    t = t.str.replace(r'\bvery\s*low\b', 'very low', regex=True)
    return t

# 3) Convert common binned numeric survey answers (hours/week, sleep hours) to numeric midpoints
def pick_col_by_keywords(columns: list[str], keywords: list[str]) -> str | None:
    cols_low = [(c, c.lower()) for c in columns]
    for c, cl in cols_low:
        if all(k in cl for k in keywords):
            return c
    return None


def midpoint_from_text(x: object) -> float:
    if pd.isna(x):
        return float('nan')
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x).lower()
    s = re.sub(r'\s+', ' ', s).strip()
    # handle 'more than 41' etc.
    if 'more than' in s:
        nums = re.findall(r'\d+', s)
        if nums:
            return float(nums[0]) + 4.0
    nums = re.findall(r'\d+(?:\.\d+)?', s)
    if len(nums) >= 2:
        return (float(nums[0]) + float(nums[1])) / 2.0
    if len(nums) == 1:
        return float(nums[0])
    return float('nan')


study_col = pick_col_by_keywords(df_model.columns.tolist(), ['hours', 'per', 'week', 'stud'])
if study_col is None:
    # fallback for slightly different phrasing in standardized headers
    study_col = pick_col_by_keywords(df_model.columns.tolist(), ['hours', 'per', 'week'])
if study_col is not None:
    df_model['study_hours_per_week_numeric'] = df_model[study_col].map(midpoint_from_text)
    print('Added study_hours_per_week_numeric from:', study_col)
else:
    print('Study-hours column not found; skipped study_hours_per_week_numeric.')

sleep_col = pick_col_by_keywords(df_model.columns.tolist(), ['hours', 'sleep'])
if sleep_col is not None:
    df_model['sleep_hours_numeric'] = df_model[sleep_col].map(midpoint_from_text)
    print('Added sleep_hours_numeric from:', sleep_col)
else:
    print('Sleep-hours column not found; skipped sleep_hours_numeric.')

# 4) Obstacles: convert ranked obstacle columns (1..8) into top-3 unique + multi-hot
def is_obstacle_rank_col(col: str) -> bool:
    c = str(col).lower().strip()
    # standardized headers often look like '1 first', '2 second', ...
    return bool(re.match(r'^[1-8]\s', c)) and any(w in c for w in ['first','second','third','forth','fourth','fifth','sixth','seventh','eighth'])


obstacle_cols = [c for c in df_model.columns if is_obstacle_rank_col(c)]
obstacle_cols = sorted(
    obstacle_cols,
    key=lambda x: int(re.match(r'^([1-8])\s', str(x).lower()).group(1)) if re.match(r'^([1-8])\s', str(x).lower()) else 99,
)
print('Detected obstacle ranking columns:', len(obstacle_cols))
if obstacle_cols:
    def top_unique_obstacles(row: pd.Series) -> str:
        seen = []
        for c in obstacle_cols:
            v = row.get(c, None)
            if pd.isna(v):
                continue
            vv = str(v).strip()
            if not vv or vv == 'ما':
                continue
            if vv not in seen:
                seen.append(vv)
            if len(seen) >= 3:
                break
        return '; '.join(seen)

    df_model['top_obstacles'] = df_model.apply(top_unique_obstacles, axis=1)
    df_model = multi_hot_expand(df_model, 'top_obstacles')
else:
    print('No obstacle ranking columns detected; skipped obstacle features.')

# Ordinal encoding for ordered words (including module level scales)
ordered_word_maps = {
    ('never', 'rarely', 'sometimes', 'often', 'always'): {k: i for i, k in enumerate(['never','rarely','sometimes','often','always'], start=1)},
    ('very low', 'low', 'medium', 'high', 'very high'): {k: i for i, k in enumerate(['very low','low','medium','high','very high'], start=1)},
    ('low', 'medium', 'high'): {k: i for i, k in enumerate(['low','medium','high'], start=1)},
}


def try_encode_ordinal_series(s: pd.Series) -> tuple[bool, pd.Series]:
    if s.dtype == 'object' or str(s.dtype) == 'string':
        norm = normalize_ordinal_token_series(s)
        vals = norm.dropna()
        if vals.empty:
            return False, s
        uniq = vals.unique()
        if len(uniq) > 30:
            return False, s

        for words, mapping in ordered_word_maps.items():
            if set(uniq).issubset(set(words)):
                encoded = norm.map(mapping).astype('float')
                return True, encoded

        sample = vals.head(200)
        hit_rate = sample.str.contains(r'\d', regex=True).mean()
        if float(hit_rate) >= 0.70:
            extracted = s.astype('string').str.extract(r'(\d+(?:\.\d+)?)', expand=False)
            encoded = pd.to_numeric(extracted, errors='coerce')
            if float(encoded.notna().mean()) >= 0.60:
                return True, encoded
    return False, s


ordinal_encoded_cols = []
for col in df_model.columns:
    if col in {'record_id', year_col, 'year_ordinal'}:
        continue
    if pd.api.types.is_numeric_dtype(df_model[col]):
        continue
    ok, enc = try_encode_ordinal_series(df_model[col])
    if ok:
        new_col = f"{col}__ordinal"
        df_model[new_col] = enc
        ordinal_encoded_cols.append(new_col)

print('Added ordinal-encoded helper columns:', len(ordinal_encoded_cols))

# Basic validations
assert df_clean.columns.is_unique, 'df_clean has duplicate columns'
assert df_model.columns.is_unique, 'df_model has duplicate columns'
assert df_clean['record_id'].is_unique, 'record_id must be unique'

# Save outputs
out_full_cleaned = data_dir / 'ensia_full_cleaned.csv'
df_clean.to_csv(out_full_cleaned, index=False)
print('Saved cleaned:', out_full_cleaned)

out_full_model = data_dir / 'ensia_full_model_ready.csv'
df_model.to_csv(out_full_model, index=False)
print('Saved model-ready:', out_full_model)


Detected year column: in which year of your studies are you currently enrolled
Detected multi-choice columns: 3


,multi_choice_cols_sample
0,what learning methods do help you understand b...
1,what types of resources do you use most
2,other problems that you face


Added study_hours_per_week_numeric from: how many hours per week do you spend studying personnal efforts
Added sleep_hours_numeric from: how many hours do you sleep
Detected obstacle ranking columns: 8
Added ordinal-encoded helper columns: 7
Saved cleaned: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_full_cleaned.csv
Saved model-ready: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_full_model_ready.csv


In [ ]:
# -------------------------

# Folder hygiene: move processed artifacts into data/processed

# -------------------------



for fname in ['ensia_full_cleaned.csv', 'ensia_full_model_ready.csv']:

    src = data_dir / fname

    dst = processed_dir / fname

    if src.exists():

        if dst.exists():

            dst.unlink()

        src.rename(dst)

        print('Moved processed artifact to:', dst)

    else:

        print('Processed artifact not found at root (ok):', src)


## 14) Split cleaned dataset by academic year



Why per-year CSVs are needed:



- ENSIA curricula differ by year (different modules/grades).

- Year-wise datasets simplify year-specific analysis and reduce irrelevant missing columns.



How we split (IMPORTANT):



- We **do not** rely on the self-reported "currently enrolled year" to decide membership.

- Instead, a row belongs to the `1Y` dataset **if it has any 1Y module/grade columns filled** (same for `2Y/3Y`).

- This means **a single student can appear in multiple year datasets** if they have multiple years of grades filled.



Outputs:



- `ensia_cleaned_1Y.csv`, `ensia_cleaned_2Y.csv`, `ensia_cleaned_3Y.csv` (if rows exist)


In [ ]:
def find_year_column(columns: list[str]) -> str:

    candidates = [

        c

        for c in columns

        if 'year of your studies' in c.lower() or 'currently enrolled' in c.lower()

    ]

    if not candidates:

        raise ValueError('Could not find a year/enrollment column. Please set it manually.')

    return candidates[0]





# -------------------------

# IMPORTANT: Split from disk artifact

# We split directly from the saved cleaned dataset.

# -------------------------

full_cleaned_path = processed_dir / 'ensia_full_cleaned.csv'

if not full_cleaned_path.exists():

    raise FileNotFoundError(

        f'Missing cleaned file: {full_cleaned_path}. Run the save-output cell first.'

    )



df_split_source = pd.read_csv(full_cleaned_path)

print('Splitting from file:', full_cleaned_path)

print('df_split_source shape:', df_split_source.shape)



# We still detect and normalize the year column (useful for diagnostics),

# but split membership is based on filled year-grade/module columns.

year_col = find_year_column(df_split_source.columns.tolist())

print('Detected year column (diagnostics only):', year_col)



# Ensure year labels are normalized (idempotent; normalize_year_value is defined earlier)

if 'normalize_year_value' not in globals():

    raise RuntimeError(

        'normalize_year_value is not defined. Run the year-normalization cell before this cell.'

    )



df_split_source[year_col] = df_split_source[year_col].map(normalize_year_value)



year_counts = df_split_source[year_col].value_counts(dropna=False)

print('Year distribution (after normalization):')

display(year_counts.to_frame('count'))



allowed_years = {'First year', 'Second year', 'Third year', 'Fourth year'}

unexpected_years = set(df_split_source[year_col].dropna().unique()) - allowed_years

if unexpected_years:

    print(

        'WARNING: unexpected year labels detected:',

        sorted(map(str, unexpected_years))[:20],

    )





def is_year_specific_col(col: str) -> bool:

    # Matches year-prefixed academic columns like "1y ...", "2y_...", ..., "4y ..."

    return bool(re.match(r'^[1234]y(?:\s|_|$)', str(col).lower()))





def cols_for_year(df: pd.DataFrame, year_tag: str) -> list[str]:

    tag = year_tag.lower()

    return [

        c

        for c in df.columns

        if bool(re.match(fr'^{re.escape(tag)}(?:\s|_|$)', str(c).lower()))

    ]





def keep_only_year_cols(df: pd.DataFrame, year_tag: str) -> list[str]:

    year_cols = cols_for_year(df, year_tag)

    general_cols = [c for c in df.columns if not is_year_specific_col(c)]

    return general_cols + year_cols





def row_has_any_filled_value(df: pd.DataFrame, cols: list[str]) -> pd.Series:

    if not cols:

        return pd.Series(False, index=df.index)



    sub = df[cols]



    # Treat NaN as missing; treat empty/whitespace-only strings as missing too.

    present = sub.notna()

    for c in cols:

        s = sub[c]

        s_str = s.astype(str)

        present[c] = present[c] & s_str.str.strip().ne('')



    return present.any(axis=1)





def make_year_splits_by_filled_grades(

    source_df: pd.DataFrame,

) -> tuple[dict[str, pd.DataFrame], pd.DataFrame]:

    y1_cols = cols_for_year(source_df, '1y')

    y2_cols = cols_for_year(source_df, '2y')

    y3_cols = cols_for_year(source_df, '3y')



    has_1y = row_has_any_filled_value(source_df, y1_cols)

    has_2y = row_has_any_filled_value(source_df, y2_cols)

    has_3y = row_has_any_filled_value(source_df, y3_cols)



    df_presence = pd.DataFrame(

        {

            'has_1y': has_1y,

            'has_2y': has_2y,

            'has_3y': has_3y,

        }

    )



    splits = {

        '1Y': source_df.loc[has_1y, keep_only_year_cols(source_df, '1y')].copy(),

        '2Y': source_df.loc[has_2y, keep_only_year_cols(source_df, '2y')].copy(),

        '3Y': source_df.loc[has_3y, keep_only_year_cols(source_df, '3y')].copy(),

    }



    return splits, df_presence





# 1) Cleaned year splits (membership by filled year-grade/module columns)

year_dfs, df_presence = make_year_splits_by_filled_grades(df_split_source)



df_1y, df_2y, df_3y = (

    year_dfs['1Y'],

    year_dfs['2Y'],

    year_dfs['3Y'],

)



print('Rows with any 1Y grades:', int(df_presence['has_1y'].sum()), '| Output cols:', df_1y.shape[1])

print('Rows with any 2Y grades:', int(df_presence['has_2y'].sum()), '| Output cols:', df_2y.shape[1])

print('Rows with any 3Y grades:', int(df_presence['has_3y'].sum()), '| Output cols:', df_3y.shape[1])



overlap_counts = df_presence.sum(axis=1).value_counts().sort_index()

print('How many year-datasets each student belongs to (0..3):')

display(overlap_counts.to_frame('row_count'))



out_1y = splits_dir / 'ensia_cleaned_1Y.csv'

out_2y = splits_dir / 'ensia_cleaned_2Y.csv'

out_3y = splits_dir / 'ensia_cleaned_3Y.csv'



if len(df_1y) > 0:

    df_1y.to_csv(out_1y, index=False)

    print('Saved:', out_1y)

else:

    print('No rows with 1Y grades; file not saved.')



if len(df_2y) > 0:

    df_2y.to_csv(out_2y, index=False)

    print('Saved:', out_2y)

else:

    print('No rows with 2Y grades; file not saved.')



if len(df_3y) > 0:

    df_3y.to_csv(out_3y, index=False)

    print('Saved:', out_3y)

else:

    print('No rows with 3Y grades; file not saved.')





# 2) Model-ready year splits (optional)

EXPORT_MODEL_READY_SPLITS = False



if EXPORT_MODEL_READY_SPLITS:

    raise RuntimeError(

        'Model-ready split export is currently disabled in this notebook. '

        'Enable it only if you really need model-ready per-year files.'

    )


Splitting from file: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_full_cleaned.csv
df_split_source shape: (475, 103)
Detected year column (diagnostics only): in which year of your studies are you currently enrolled
Year distribution (after normalization):


,count
in which year of your studies are you currently enrolled,
First year,215
Second year,164
Third year,81
Fourth year,15


Rows with any 1Y grades: 210 | Output cols: 58
Rows with any 2Y grades: 101 | Output cols: 64
Rows with any 3Y grades: 12 | Output cols: 49
How many year-datasets each student belongs to (0..3):


,row_count
0,215
1,209
2,39
3,12


Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_1Y.csv
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_2Y.csv
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_3Y.csv


In [ ]:
# -------------------------

# Post-split: Group module columns + compute per-group averages (NON-imputed splits)

# -------------------------

# Goal: reduce sparsity by aggregating many module columns into a few group scores.

# - Convert ordinal text (very low..very high) -> numeric.

# - Compute mean per group, ignoring missing modules (NaN/empty).

# - Drop the individual module columns (keep grouped features + counts).



RUN_MODULE_GROUPING = True

EXPORT_GROUPED_SPLITS = True



# If True, the exported *_grouped.csv will NOT contain the per-module columns.

DROP_GROUPED_SOURCE_MODULE_COLUMNS = True



# Make sure group averages stay on a 0..5-like scale.

# (With ORDINAL_MAP 1..5, group averages naturally fall in [1,5].)

CLIP_GROUP_AVGS_TO_1_5 = True



ORDINAL_MAP = {

    'very low': 1,

    'low': 2,

    'medium': 3,

    'moderate': 3,

    'high': 4,

    'very high': 5,

}



MATH_KEYWORDS = [

    'math', 'mathemat', 'analysis', 'algebra', 'linear', 'lin alg', 'prob', 'proba', 'stat',

    'inference', 'numerical', 'optim', 'optimis', 'operations research', 'stochastic', 'time series',

    'logic',

]



PROG_KEYWORDS = [

    'prog', 'program', 'oop', 'dsa', 'algorithm', 'linux', 'operating system', 'oper syst',

    'database', 'web', 'software', 'mobile', 'network', 'security', 'architecture', 'archi',

    'ai', 'machine learning', 'data mining', 'theory of computing',

]



def _norm_text(x: object) -> str | None:

    if pd.isna(x):

        return None

    s = str(x).strip().lower()

    s = ' '.join(s.split())

    return s if s else None



def detect_ordinal_module_cols(df_year: pd.DataFrame, year_tag: str) -> list[str]:

    # Restrict to that year’s module columns (prefix) to avoid touching unrelated survey columns.

    prefix = year_tag.lower() + ' '

    obj_cols = df_year.select_dtypes(include=['object', 'string']).columns.tolist()

    candidates = [c for c in obj_cols if str(c).lower().startswith(prefix)]



    tokens = set(ORDINAL_MAP.keys())

    out = []

    for c in candidates:

        vals = df_year[c].dropna().map(_norm_text)

        vals = vals[vals.notna()]

        if len(vals) == 0:

            continue

        uniq = set(vals.unique().tolist())

        overlap = uniq & tokens

        # Heuristic: accept if most distinct values look like our ordinal scale

        if overlap and (len(overlap) / max(1, len(uniq)) >= 0.5):

            out.append(c)

    return out



def classify_module(col_name: str) -> str:

    name = _norm_text(col_name) or ''

    # Prefer explicit math matches first

    if any(k in name for k in MATH_KEYWORDS):

        return 'math'

    if any(k in name for k in PROG_KEYWORDS):

        return 'programming'

    return 'other'



def ordinal_to_numeric_series(s: pd.Series) -> pd.Series:

    # Convert empty strings -> NA, then map ordinal text -> numeric

    s_txt = s.astype('string')

    s_txt = s_txt.where(s_txt.str.strip().ne(''), pd.NA)

    mapped = s_txt.map(lambda x: ORDINAL_MAP.get(_norm_text(x), pd.NA))

    return pd.to_numeric(mapped, errors='coerce')



def add_group_averages(

    df_year: pd.DataFrame,

    year_tag: str,

    *,

    drop_source_module_columns: bool,

) -> tuple[pd.DataFrame, dict[str, list[str]]]:

    df_out = df_year.copy()

    module_cols = detect_ordinal_module_cols(df_out, year_tag)



    groups = {'math': [], 'programming': [], 'other': []}

    for c in module_cols:

        groups[classify_module(c)].append(c)



    # Convert module cols to numeric scores in a temporary frame

    score_frame = pd.DataFrame(index=df_out.index)

    for c in module_cols:

        score_frame[c] = ordinal_to_numeric_series(df_out[c])



    def _avg_and_count(cols: list[str]) -> tuple[pd.Series, pd.Series]:

        if not cols:

            return (

                pd.Series(pd.NA, index=df_out.index, dtype='float'),

                pd.Series(0, index=df_out.index, dtype='int'),

            )

        sub = score_frame[cols]

        cnt = sub.notna().sum(axis=1).astype('int')

        ssum = sub.sum(axis=1, skipna=True)

        avg = (ssum / cnt.where(cnt.ne(0), pd.NA)).astype('float')

        return avg, cnt



    math_avg, math_n = _avg_and_count(groups['math'])

    prog_avg, prog_n = _avg_and_count(groups['programming'])

    other_avg, other_n = _avg_and_count(groups['other'])

    all_avg, all_n = _avg_and_count(module_cols)



    if CLIP_GROUP_AVGS_TO_1_5:

        math_avg = math_avg.clip(lower=1, upper=5)

        prog_avg = prog_avg.clip(lower=1, upper=5)

        other_avg = other_avg.clip(lower=1, upper=5)

        all_avg = all_avg.clip(lower=1, upper=5)



    df_out[f'{year_tag}_math_avg'] = math_avg

    df_out[f'{year_tag}_math_n'] = math_n

    df_out[f'{year_tag}_programming_avg'] = prog_avg

    df_out[f'{year_tag}_programming_n'] = prog_n

    df_out[f'{year_tag}_other_avg'] = other_avg

    df_out[f'{year_tag}_other_n'] = other_n

    df_out[f'{year_tag}_all_modules_avg'] = all_avg

    df_out[f'{year_tag}_all_modules_n'] = all_n



    if drop_source_module_columns:

        df_out = df_out.drop(columns=module_cols, errors='ignore')



    return df_out, groups



if RUN_MODULE_GROUPING:

    df_1y_grouped, g1 = add_group_averages(

        df_1y,

        '1Y',

        drop_source_module_columns=DROP_GROUPED_SOURCE_MODULE_COLUMNS,

    )

    df_2y_grouped, g2 = add_group_averages(

        df_2y,

        '2Y',

        drop_source_module_columns=DROP_GROUPED_SOURCE_MODULE_COLUMNS,

    )

    df_3y_grouped, g3 = add_group_averages(

        df_3y,

        '3Y',

        drop_source_module_columns=DROP_GROUPED_SOURCE_MODULE_COLUMNS,

    )



    print('Grouped module features added (non-imputed splits).')

    print('1Y module cols grouped:', {k: len(v) for k, v in g1.items()})

    print('2Y module cols grouped:', {k: len(v) for k, v in g2.items()})

    print('3Y module cols grouped:', {k: len(v) for k, v in g3.items()})

    print('Grouped shapes:', '1Y', df_1y_grouped.shape, '| 2Y', df_2y_grouped.shape, '| 3Y', df_3y_grouped.shape)



    if EXPORT_GROUPED_SPLITS:

        out_1y_grp = grouped_dir / 'ensia_cleaned_1Y_grouped.csv'

        out_2y_grp = grouped_dir / 'ensia_cleaned_2Y_grouped.csv'

        out_3y_grp = grouped_dir / 'ensia_cleaned_3Y_grouped.csv'

        df_1y_grouped.to_csv(out_1y_grp, index=False)

        df_2y_grouped.to_csv(out_2y_grp, index=False)

        df_3y_grouped.to_csv(out_3y_grp, index=False)

        print('Saved:', out_1y_grp)

        print('Saved:', out_2y_grp)

        print('Saved:', out_3y_grp)


Grouped module features added (non-imputed splits).
1Y module cols grouped: {'math': 8, 'programming': 4, 'other': 9}
2Y module cols grouped: {'math': 8, 'programming': 13, 'other': 6}
3Y module cols grouped: {'math': 4, 'programming': 7, 'other': 1}
Grouped shapes: 1Y (210, 45) | 2Y (101, 45) | 3Y (12, 45)
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_1Y_grouped.csv
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_2Y_grouped.csv
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_3Y_grouped.csv


In [ ]:
# -------------------------

# Imputation (GROUPED splits): median for numeric, mode for non-numeric

# -------------------------

# Requirement: starting from the grouped datasets, replace missing values using median.

# Note: median is defined for numeric columns; for non-numeric columns we use mode so we can eliminate missingness.



RUN_GROUPED_IMPUTATION = True

EXPORT_GROUPED_IMPUTED_SPLITS = True



# If True, any non-numeric (object/string) columns will be filled with their most frequent value (mode).

IMPUTE_NON_NUMERIC_WITH_MODE = True



def _normalize_blank_strings_to_na(df_in: pd.DataFrame) -> pd.DataFrame:

    df_out = df_in.copy()

    for c in df_out.columns:

        if df_out[c].dtype == object or str(df_out[c].dtype).startswith('string'):

            s = df_out[c].astype('string')

            df_out[c] = s.where(s.str.strip().ne(''), pd.NA)

    return df_out



def impute_grouped_median(df_in: pd.DataFrame) -> pd.DataFrame:

    df_out = _normalize_blank_strings_to_na(df_in)



    # Numeric: median

    num_cols = df_out.select_dtypes(include=[np.number, 'bool']).columns.tolist()

    for c in num_cols:

        med = df_out[c].median(skipna=True)

        if pd.notna(med):

            df_out[c] = df_out[c].fillna(med)



    # Non-numeric: mode (optional)

    if IMPUTE_NON_NUMERIC_WITH_MODE:

        non_num_cols = [c for c in df_out.columns if c not in set(num_cols)]

        for c in non_num_cols:

            mode = df_out[c].mode(dropna=True)

            fill = mode.iloc[0] if len(mode) else 'Unknown'

            df_out[c] = df_out[c].fillna(fill)



    return df_out



def _missing_cells(df_in: pd.DataFrame) -> int:

    df_tmp = _normalize_blank_strings_to_na(df_in)

    return int(df_tmp.isna().sum().sum())



if RUN_GROUPED_IMPUTATION:

    df_1y_grouped_imputed = impute_grouped_median(df_1y_grouped)

    df_2y_grouped_imputed = impute_grouped_median(df_2y_grouped)

    df_3y_grouped_imputed = impute_grouped_median(df_3y_grouped)



    print('Grouped imputation done (median for numeric; mode for non-numeric=', IMPUTE_NON_NUMERIC_WITH_MODE, ').')

    print('Remaining missing cells:',

          '1Y', _missing_cells(df_1y_grouped_imputed),

          '| 2Y', _missing_cells(df_2y_grouped_imputed),

          '| 3Y', _missing_cells(df_3y_grouped_imputed))



    if EXPORT_GROUPED_IMPUTED_SPLITS:

        out_1y_grp_imp = grouped_imputed_dir / 'ensia_cleaned_1Y_grouped_imputed.csv'

        out_2y_grp_imp = grouped_imputed_dir / 'ensia_cleaned_2Y_grouped_imputed.csv'

        out_3y_grp_imp = grouped_imputed_dir / 'ensia_cleaned_3Y_grouped_imputed.csv'

        df_1y_grouped_imputed.to_csv(out_1y_grp_imp, index=False)

        df_2y_grouped_imputed.to_csv(out_2y_grp_imp, index=False)

        df_3y_grouped_imputed.to_csv(out_3y_grp_imp, index=False)

        print('Saved:', out_1y_grp_imp)

        print('Saved:', out_2y_grp_imp)

        print('Saved:', out_3y_grp_imp)


Grouped imputation done (median for numeric; mode for non-numeric= True ).
Remaining missing cells: 1Y 0 | 2Y 0 | 3Y 0
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_1Y_grouped_imputed.csv
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_2Y_grouped_imputed.csv
Saved: C:\Users\hp\Desktop\data_mining\ENSIA-students-performance\ensia_cleaned_3Y_grouped_imputed.csv


In [271]:
# -------------------------

# Emptiness report (GROUPED splits)

# -------------------------

# If grouped-imputed splits exist, report on those; else report on grouped splits.



def emptiness_report(df_in: pd.DataFrame) -> pd.DataFrame:

    df = df_in.copy()



    empty_pct = []

    for c in df.columns:

        if pd.api.types.is_numeric_dtype(df[c]):

            missing = df[c].isna()

        else:

            s = df[c].astype('string')

            missing = s.isna() | (s.str.strip() == '')

        empty_pct.append(missing.mean() * 100)



    rep = pd.DataFrame({'column': df.columns, 'empty_pct': empty_pct}).sort_values('empty_pct', ascending=False)

    return rep



use_1y = df_1y_grouped_imputed if 'df_1y_grouped_imputed' in globals() else df_1y_grouped

use_2y = df_2y_grouped_imputed if 'df_2y_grouped_imputed' in globals() else df_2y_grouped

use_3y = df_3y_grouped_imputed if 'df_3y_grouped_imputed' in globals() else df_3y_grouped



grouped_year_dfs = {

    '1Y_grouped': use_1y,

    '2Y_grouped': use_2y,

    '3Y_grouped': use_3y,

}



for name, dfy in grouped_year_dfs.items():

    rep = emptiness_report(dfy)

    print('\n' + '=' * 90)

    print(f"{name} emptiness report | rows={len(dfy)} cols={dfy.shape[1]}\n")

    display(rep.head(40))



    gt50 = int((rep['empty_pct'] >= 50).sum())

    gt90 = int((rep['empty_pct'] >= 90).sum())

    full = int((rep['empty_pct'] == 100).sum())

    print(f"Columns with empty_pct >= 50%: {gt50}")

    print(f"Columns with empty_pct >= 90%: {gt90}")

    print(f"Columns with empty_pct = 100%: {full}")



1Y_grouped emptiness report | rows=210 cols=45



,column,empty_pct
0,record_id,0.0000
1,what is your gender,0.0000
2,in which year of your studies are you currentl...,0.0000
3,what was your bac specialty,0.0000
4,how many hours per week do you spend studying ...,0.0000
5,where do you prefer to study,0.0000
6,you prefer to study,0.0000
7,what learning methods do help you understand b...,0.0000
8,what types of resources do you use most,0.0000
9,do you use a planer or a schedule in your revi...,0.0000


Columns with empty_pct >= 50%: 0
Columns with empty_pct >= 90%: 0
Columns with empty_pct = 100%: 0

2Y_grouped emptiness report | rows=101 cols=45



,column,empty_pct
0,record_id,0.0000
1,what is your gender,0.0000
2,in which year of your studies are you currentl...,0.0000
3,what was your bac specialty,0.0000
4,how many hours per week do you spend studying ...,0.0000
5,where do you prefer to study,0.0000
6,you prefer to study,0.0000
7,what learning methods do help you understand b...,0.0000
8,what types of resources do you use most,0.0000
9,do you use a planer or a schedule in your revi...,0.0000


Columns with empty_pct >= 50%: 0
Columns with empty_pct >= 90%: 0
Columns with empty_pct = 100%: 0

3Y_grouped emptiness report | rows=12 cols=45



,column,empty_pct
0,record_id,0.0000
1,what is your gender,0.0000
2,in which year of your studies are you currentl...,0.0000
3,what was your bac specialty,0.0000
4,how many hours per week do you spend studying ...,0.0000
5,where do you prefer to study,0.0000
6,you prefer to study,0.0000
7,what learning methods do help you understand b...,0.0000
8,what types of resources do you use most,0.0000
9,do you use a planer or a schedule in your revi...,0.0000


Columns with empty_pct >= 50%: 0
Columns with empty_pct >= 90%: 0
Columns with empty_pct = 100%: 0


## 15) Per-year output columns (attributes)



This section lists the **exact columns present in each per-year exported dataset** (`ensia_cleaned_1Y.csv` ... `ensia_cleaned_3Y.csv`).


In [272]:
# Columns (attributes) per year dataset (as exported)
year_dfs = {
    '1Y': df_1y,
    '2Y': df_2y,
    '3Y': df_3y,
}

year_cols_rows = []
for year_label, df_year in year_dfs.items():
    cols = df_year.columns.tolist()
    year_cols_rows.append({
        'year': year_label,
        'n_rows': int(len(df_year)),
        'n_cols': int(len(cols)),
        'columns': ', '.join(cols),
    })

year_columns_report = pd.DataFrame(year_cols_rows).sort_values('year')
display(year_columns_report)

# Also display as separate lists (easier to copy/paste)
for year_label, df_year in year_dfs.items():
    print('\n' + '=' * 80)
    print(f'{year_label} columns ({len(df_year.columns)}):')
    for c in df_year.columns:
        print(' -', c)


,year,n_rows,n_cols,columns
0,1Y,210,58,"record_id, what is your gender, in which year ..."
1,2Y,101,64,"record_id, what is your gender, in which year ..."
2,3Y,12,49,"record_id, what is your gender, in which year ..."



1Y columns (58):
 - record_id
 - what is your gender
 - in which year of your studies are you currently enrolled
 - what was your bac specialty
 - how many hours per week do you spend studying personnal efforts
 - where do you prefer to study
 - you prefer to study
 - what learning methods do help you understand better
 - what types of resources do you use most
 - do you use a planer or a schedule in your revisions
 - for the next 8 questions you will have a list of problems obstacles classify them according to how much they influence your studies 1 first
 - 2 second
 - 3 third
 - 4 fourth
 - 5 fifth
 - 6 sixth
 - 7 seventh
 - 8 eighth
 - other problems that you face
 - rate your satisfaction with the school s program
 - do you engage in any external activities not related to the study field
 - your level of motivation for joining ensia
 - how much do you like the field of artificial intelligence
 - how do other s feedback influence you
 - how many hours do you sleep
 - how well do yo

## 15a) Per-year missingness (emptiness) report

For each per-year dataset, we compute emptiness per column (same definition as before):
- Numeric: empty = `NaN`
- Categorical: empty = `NaN` or blank string

Tables are sorted by `empty_pct` descending.

In [273]:
def emptiness_report(df: pd.DataFrame) -> pd.DataFrame:
    n = int(len(df))
    rows = []
    for col in df.columns:
        s = df[col]
        is_num = pd.api.types.is_numeric_dtype(s)
        if is_num:
            empty_mask = s.isna()
            kind = 'numeric'
        else:
            s_txt = s.astype('string')
            empty_mask = s_txt.isna() | (s_txt.str.strip() == '')
            kind = 'categorical'
        empty_count = int(empty_mask.sum())
        rows.append({
            'column': col,
            'kind': kind,
            'dtype': str(s.dtype),
            'empty_count': empty_count,
            'empty_pct': (empty_count / n) if n else float('nan'),
            'non_empty_count': int(n - empty_count),
        })
    return (
        pd.DataFrame(rows)
        .sort_values(['empty_pct', 'empty_count', 'column'], ascending=[False, False, True])
        .reset_index(drop=True)
    )


year_dfs = {
    '1Y': df_1y,
    '2Y': df_2y,
    '3Y': df_3y,
}

for y, dfy in year_dfs.items():
    print('\n' + '=' * 90)
    print(f'{y} emptiness report | rows={len(dfy)} cols={dfy.shape[1]}')
    rep = emptiness_report(dfy)
    display(rep)



1Y emptiness report | rows=210 cols=58


,column,kind,dtype,empty_count,empty_pct,non_empty_count
0,1y a com i,categorical,object,132,0.6286,78
1,1y a com ii,categorical,object,132,0.6286,78
2,1y algebra,categorical,object,132,0.6286,78
3,1y analysis i,categorical,object,132,0.6286,78
4,1y analysis ii,categorical,object,132,0.6286,78
5,1y intensive english,categorical,object,132,0.6286,78
6,1y intro prog c,categorical,object,132,0.6286,78
7,1y proba stat,categorical,object,132,0.6286,78
8,1y seminar,categorical,object,132,0.6286,78
9,1y introduction to linux,categorical,object,129,0.6143,81



2Y emptiness report | rows=101 cols=64


,column,kind,dtype,empty_count,empty_pct,non_empty_count
0,2y electronic circuits labs,categorical,object,74,0.7327,27
1,2y dsa 2,categorical,object,73,0.7228,28
2,2y introduction to ai,categorical,object,73,0.7228,28
3,2y introduction to business,categorical,object,73,0.7228,28
4,2y mathematical analysis 3,categorical,object,73,0.7228,28
5,2y operating systems,categorical,object,73,0.7228,28
6,2y probability,categorical,object,73,0.7228,28
7,2y statistical inference,categorical,object,73,0.7228,28
8,2y web development,categorical,object,73,0.7228,28
9,2y computer architecture,categorical,object,72,0.7129,29



3Y emptiness report | rows=12 cols=49


,column,kind,dtype,empty_count,empty_pct,non_empty_count
0,3y advanced databases,categorical,object,8,0.6667,4
1,3y computer and network security,categorical,object,8,0.6667,4
2,3y data mining,categorical,object,8,0.6667,4
3,3y entrepreneurship and innovation,categorical,object,8,0.6667,4
4,3y machine learning,categorical,object,8,0.6667,4
5,3y mobile development,categorical,object,8,0.6667,4
6,3y networks and protocols,categorical,object,8,0.6667,4
7,3y numerical methods and optimisation,categorical,object,8,0.6667,4
8,3y operations research,categorical,object,8,0.6667,4
9,3y software engineering,categorical,object,8,0.6667,4
